Codebase: https://www.kaggle.com/code/kartikbatra/multilabelclassification/notebook

 # Import Packages and Dataset

In [ ]:
# Import libraries 
import os,sys
import torch
import time
import torch.nn as nn
# from torch.utils.data import Dataset,DataLoader
# from sklearn.metrics import f1_score,precision_score
# from PIL import Image
from torchvision.transforms import transforms 
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
# import torch.optim as optim
# from torch.optim import lr_scheduler
# from pathlib import Path
# import math
# import copy
from functools import partial
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Path Setting
**NOTE** Comment uncomment LINE No 4,5 to change dataset path

In [ ]:
path_curr = os.getcwd()
path_notebook = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

#path_dataset = os.path.abspath(os.path.join(path_notebook,'dataset/CelebA/CelebAMask-HQ'))
path_dataset = 'D:\\DS\\CelebAMask-HQ\\'

path_main = path_curr
results_dir  = path_curr


# path_celeba_main    = os.path.join(path_notebook, 'E7_CelebA')

ds_imgs_path = os.path.join(path_dataset, 'CelebA-HQ-img')
ds_anno_path = os.path.join(path_dataset, 'CelebAMask-HQ-mask-anno')
path_train_weights = os.path.join(path_curr, 'models/model.pth')
path_repos     = os.path.join(path_notebook,'utils')

print(f'codes path:        {os.path.exists(path_curr)} | {path_curr}')
print(f'models paths:      {os.path.exists(path_train_weights)} | {path_train_weights}')
print('-'*120)
print(f'dataset path:      {os.path.exists(path_dataset)} | {path_dataset}')
print(f'dataset imgs :     {os.path.exists(ds_imgs_path)} | {ds_imgs_path}')
print(f'dataset anno:      {os.path.exists(ds_anno_path)} | {ds_anno_path}')

# Verify Load Model 

In [ ]:
print('='*120)
print('Trained Weights Exists? :\t',os.path.exists(path_train_weights))
print('='*120)

In [ ]:
att_csv_celeba = False
csv_anno_file = f'{path_dataset}//CelebA-HQ-attribute.txt'
df_attr = pd.read_csv(csv_anno_file, delim_whitespace=True,skiprows=[0])
df_attr.replace(-1,0,inplace=True)
df_attr.head(2)
print(len(np.unique(df_attr.image_id)))

In [ ]:
# # Get Dataset
# path_ds = 'D://DS//archive'
# DS_path = f'{path_ds}/img_align_celeba/img_align_celeba'

# image_path=f'{path_ds}/img_align_celeba/img_align_celeba'
# if att_csv_celeba:
#     pass
#     # df_attr=pd.read_csv(f'{path_ds}/list_attr_celeba.csv')
#     # df_landmarks=pd.read_csv(f'{path_ds}/list_landmarks_align_celeba.csv')
#     # df_bbox=pd.read_csv(f'{path_ds}/list_bbox_celeba.csv')
#     # df_attr.head()
#     # df_attr.replace(-1,0,inplace=True)
#     # df_attr.head()
#     # print(len(np.unique(df_attr.image_id)))
# else:
#     df_attr = pd.read_csv(csv_anno_file, delim_whitespace=True,skiprows=[0])
#     df_attr.replace(-1,0,inplace=True)
#     df_attr.head(2)
#     print(len(np.unique(df_attr.image_id)))

In [ ]:
df_attr.head(3)

In [ ]:
classes_names = []
for i,x in  enumerate(df_attr.columns):
    if i!=0:
        classes_names.append(x)

In [ ]:
# Visualize Label count of Dataset and check for imbalance
arr=pd.DataFrame(df_attr.iloc[:,1:].sum(axis=0))
arr.columns=['labels']
arr.sort_values(by='labels',ascending=False)
plt.figure(figsize=(16,8))
plt.bar(arr.index,arr['labels'])
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Look for correlation between labels
import seaborn as sns
plt.figure(figsize=(16,12))
sns.heatmap(df_attr.iloc[:,1:].corr(), cmap="RdYlBu", vmin=-1, vmax=1)
plt.show()

# DataLoader 

In [ ]:
# # Class to get data in specific format and preprocessing.
# class CelebDataset(Dataset):
#     def __init__(self,df_1,image_path,transform=None,mode='train'):
#         super().__init__()
#         self.attr=df_1.drop(['image_id'],axis=1)
#         self.path=image_path
#         self.image_id=df_1['image_id']
#         self.transform=transform
#         self.mode=mode
#     def __len__(self):
#         return self.image_id.shape[0]
#     def __getitem__(self,idx:int):
#         image_name=self.image_id.iloc[idx]
#         image=Image.open(os.path.join(image_path,image_name))
#         attributes=np.asarray(self.attr.iloc[idx].T,dtype=np.float32)
#         if self.transform:
#             image=self.transform(image)
#         return image,attributes    

In [ ]:
# # Split Dataset into train,test and valid
# from sklearn.model_selection import train_test_split
# train_df,test=train_test_split(df_attr,test_size=0.1,shuffle=True,random_state=212)
# valid_df,test_df=train_test_split(test,test_size=0.5,random_state=212)
# train_df.shape,valid_df.shape,test_df.shape

In [ ]:
# # Apply Data augmentation and different type of transforms on train data.
# train_transform=transforms.Compose([transforms.Resize((224,224)),transforms.RandomVerticalFlip(p=0.5),
#                                     transforms.RandomHorizontalFlip(p=0.5),transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
#                                     transforms.ToTensor(),
#                                     transforms.Normalize(mean=[0.5063, 0.4258, 0.3832],std=[0.2644, 0.2436, 0.2397])])
# Apply Data augmentation and different type of transforms on test and validation data.
valid_transform=transforms.Compose([transforms.Resize((224,224)),
                              transforms.ToTensor(),
                              transforms.Normalize(mean=[0.5063, 0.4258, 0.3832],std=[0.2644, 0.2436, 0.2397])])

In [ ]:
# # Create Dataset Object and DataLoader for train and validation set 
# train_data=CelebDataset(train_df,image_path,train_transform)
# train_loader=DataLoader(train_data,batch_size=64,shuffle=True,num_workers=4)
# valid_data=CelebDataset(valid_df,image_path,valid_transform)
# valid_loader=DataLoader(valid_data,batch_size=64,num_workers=4)

# Visualizing Data

In [ ]:
# # function to visualize dataset
# def imshow(images,attr,idx:int):
#     images=images.cpu().numpy().transpose((0,2,3,1))
#     plt.imshow(images[idx]*[0.2644, 0.2436, 0.2397]+[0.5063, 0.4258, 0.3832])
#     labels=df_attr.columns.tolist()
#     labels=labels[1:]
#     att=attr[idx].numpy()
#     labels=[label for label,a in list(zip(labels,att)) if a==1]
#     plt.xlabel("\n".join(labels))
#     plt.show()

In [ ]:
# # Visualizing Data
# images,attr=next(iter(train_loader))
# imshow(images,attr,0)

# Creating Model

In [ ]:
# Defining Class for Single Layer. 
class Layer(nn.Module):
    def __init__(self,in_ch,out_ch,kernel_size=3,stride=1,padding=1):
        super(Layer,self).__init__()
        self.conv=nn.Conv2d(in_ch,out_ch,kernel_size,stride,padding)
        self.bn=nn.BatchNorm2d(out_ch)
        self.relu=nn.ReLU()
        nn.init.xavier_uniform_(self.conv.weight)
    def forward(self,Input):
        output=self.conv(Input)
        output=self.bn(output)
        output=self.relu(output)
        return output

In [ ]:
# Complete model
class CelebModel(nn.Module):
    def __init__(self,num_classes=40):
        super(CelebModel,self).__init__()
        
        self.unit1=Layer(in_ch=3,out_ch=32)        
        self.unit2=Layer(in_ch=32,out_ch=32)
        self.pool1=nn.MaxPool2d(kernel_size=2)
        
        self.unit3=Layer(in_ch=32,out_ch=64)
        self.unit4=Layer(in_ch=64,out_ch=64)
        self.pool2=nn.MaxPool2d(kernel_size=2)
        
        self.unit5=Layer(in_ch=64,out_ch=128)
        self.unit6=Layer(in_ch=128,out_ch=128)
        self.unit7=Layer(in_ch=128,out_ch=128)
        self.pool3=nn.MaxPool2d(kernel_size=2)
        
        self.unit8=Layer(in_ch=128,out_ch=256,kernel_size=5,padding=0)
        self.unit9=Layer(in_ch=256,out_ch=256,kernel_size=5,padding=0)
        self.unit10=Layer(in_ch=256,out_ch=256,kernel_size=5,padding=0)
        self.pool4=nn.MaxPool2d(kernel_size=2)
        
        self.drop2=nn.Dropout(0.5)   
        
        self.unit11=Layer(in_ch=256,out_ch=512,kernel_size=3,padding=0)
        self.unit12=Layer(in_ch=512,out_ch=512,kernel_size=3,padding=0)
        self.unit13=Layer(in_ch=512,out_ch=512,kernel_size=3,padding=0)
        
        self.pool5=nn.AvgPool2d(kernel_size=2)
        
        self.drop3=nn.Dropout(0.5)
        
        self.model=nn.Sequential(self.unit1,self.unit2,self.pool1,self.unit3,
                                 self.unit4,self.pool2,self.unit5,self.unit6,
                                 self.unit7,self.pool3,self.unit8,self.unit9,
                                 self.unit10,self.pool4,self.drop2,self.unit11,
                                 self.unit12,self.unit13,self.pool5,self.drop3)
        
        self.fc=nn.Linear(in_features=512,out_features=num_classes)
        
    def forward(self,Input):
        
        output=self.model(Input)
        output=output.view(-1,512)
        output=self.fc(output)
        
        return output

In [ ]:
# setting environment 
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Creating Model Object,loss function,optimizer and scheduler
model=CelebModel().to(device)
# criterion=nn.BCEWithLogitsLoss()
# optimizer=optim.SGD(model.parameters(),lr=0.1,momentum=0.9)
# scheduler=optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0000001, max_lr=1,mode='exp_range')

# Training and Validating Model

In [ ]:
# # Creating a Checkpoint to Save model
# def Save_model(model,cur_acc,best_acc,filename):
#     if cur_acc>best_acc:
#         best_acc=cur_acc
#         best_model=copy.deepcopy(model)
#         torch.save(best_model.state_dict(),filename)
#     return best_acc

In [ ]:
# # Create validation loop 
# def val_model():
#     model.eval()
#     epoch_loss=0
#     acc=0
#     with torch.no_grad():
#         val_pred=[]
#         val_lbl=[]
#         for images,attr in valid_loader:
            
#             if torch.cuda.is_available():
#                 images=images.to(device)
#                 attr=attr.to(device)
            
#             output=model(images)
            
#             loss=criterion(output,attr)
            
#             prediction=torch.round(torch.sigmoid(output))
            
#             epoch_loss+=loss.item()*images.size(0)
            
#             val_lbl.append(attr.detach().cpu())
            
#             val_pred.append(prediction.detach().cpu()) 
            
#         epoch_loss/=len(valid_data)
        
#         acc=f1_score(torch.cat(val_lbl,dim=0).numpy(),torch.cat(val_pred,dim=0).numpy(),average='samples')
        
#         return epoch_loss,acc    

In [ ]:
# # Create Training loop 
# def train_model(num_epochs):
#     train_loss=[]
#     best_acc=0.0
#     for i in range(num_epochs):
#         epoch_loss=0
#         acc=0
#         j=0
#         epoch_pred=[]
#         epoch_lbl=[]
#         model.train()
#         for images,attr in train_loader:
#             j+=1 
#             if torch.cuda.is_available():
#                 images=images.to(device)
#                 attr=attr.to(device)
            
#             optimizer.zero_grad()
            
#             output=model(images)
            
#             loss=criterion(output,attr)
            
#             loss.backward()
            
#             optimizer.step()
            
#             prediction=torch.round(torch.sigmoid(output))
            
#             epoch_loss+=loss.item()*images.size(0)
            
#             epoch_lbl.append(attr.detach().cpu())
            
#             epoch_pred.append(prediction.detach().cpu()) 
            
#             torch.cuda.empty_cache()
            
#             if (j+1)%500==0:
#                 print(f'Loss #{(j+1)//500}:{loss.item()}')
            
#             scheduler.step()
            
#         epoch_loss/=len(train_data)
        
#         acc=f1_score(torch.cat(epoch_lbl,dim=0).numpy(),torch.cat(epoch_pred,dim=0).numpy(),average='samples')
        
#         train_loss.append(epoch_loss)
        
#         val_loss,val_acc=val_model()
        
#         best_acc=Save_model(model,val_acc,best_acc,'model.pth')   # save model
        
#         print(f'Epoch : {i+1} , Training loss:{epoch_loss} , Training f1_score:{acc} , Validation loss:{val_loss} , Validation f1_score:{val_acc}')

In [ ]:
# # Training model
# num_epochs=7
# if __name__=='__main__':
#     train_model(num_epochs)

# Testing Model

In [ ]:
# # Create Dataset Object and DataLoader for train and validation set
# test_data=CelebDataset(test_df,image_path,valid_transform)
# test_loader=DataLoader(test_data,batch_size=64,shuffle=False,num_workers=4)

In [ ]:
# # Function to predict Labels
# def Predict(images,model,idx:int):
#     pred=model(images)
#     images=images.cpu().numpy().transpose((0,2,3,1))
#     fig = plt.figure(figsize=(15,5))
#     plt.subplot(1,2, 1)
#     plt.imshow(images[idx])
#     labels=df_attr.columns.tolist()
#     labels=labels[1:]
#     attr=torch.round(torch.sigmoid(pred)).cpu().detach()
#     prd=torch.sigmoid(pred[idx]).cpu().detach().numpy()
#     att=attr[idx].numpy()
#     labels=[label for label,a in list(zip(labels,att)) if a==1]
#     pred_list=[p for p,a in list(zip(prd,att)) if a==1]
#     plt.subplot(1,2,2)
#     plt.barh(labels,pred_list)
#     plt.show()
#     return labels

In [ ]:
# # Loading Save model
# model.load_state_dict(torch.load('D:/DS/archive/trained_models/model.pth',map_location=torch.device('cpu')))
# model.eval()

In [ ]:
# Loading Save model
model.load_state_dict(torch.load(path_train_weights,map_location=device))
model.eval()

# eXplaining Decision for face attributes

## Load Libraries for XAI

In [ ]:
from PIL import Image
import cv2,re

import shap_bpt
from tqdm.auto import tqdm
from skimage.filters import gaussian
from datetime import datetime
import warnings
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )

## Dataset Annotation Path

# Default Param

In [ ]:
image_to_explain = image_to_explain_preproc = image_to_explain_tensor = label_to_explain = predicted_fS = predicted_f0 = None
f_S = f_0 = predicted_cls =background_image_set=background_tensors=None
background_image_set = None

In [ ]:
intrested_class = 11

num_explained_classes = 1
sorted_classes = np.array([intrested_class])
print(classes_names[intrested_class], sorted_classes)

In [ ]:
exp_type = 'E5'

## Backgorund Replacement Values

In [ ]:
# background_type        = 'blurred'     # black , white , gray , noise , blurred , full
# background_type        = 'black'     # black , white , gray , noise , blurred , full
# background_type        = 'white'     # black , white , gray , noise , blurred , full
# background_type        = 'noise'     # black , white , gray , noise , blurred , full
background_type        = 'gray'     # black , white , gray , noise , blurred , full
# background_type        = 'full'     # black , white , gray , noise , blurred , full

In [ ]:
results_path_main = os.path.join(path_curr,       'results')
path_csv          = os.path.join(results_path_main,     'csv')
results_selected  = os.path.join(results_path_main,'selected')
results_path      = os.path.join(results_path_main,('bg_'+background_type))

os.makedirs(results_path,              exist_ok=True)
os.makedirs(results_selected,          exist_ok=True)
os.makedirs(results_path,              exist_ok=True)
os.makedirs(path_csv,                  exist_ok=True)
print(results_path,results_selected)

In [ ]:
batch_size  = 8
max_weight  = None
num_samples = 100
test_single = False

# Functions

## Masking Functions

In [ ]:
def f_masked_resnet(masks):
    global f, background_tensors, image_to_explain_tensor
    N,H,W = masks.shape
    B = background_tensors.shape[0]
    masks_tensor = torch.tensor(np.array(masks)).to(device)                     # N boolean masks with size W*H
    masks_tensor = torch.reshape(masks_tensor, (N,1,H,W))                       # N*H*W    -> N*1*H*W
    masks_tensor = torch.tile(masks_tensor, dims=(1,3,B,1)).reshape(B*N,3,H,W)  # N*1*H*W  -> NB*3*H*W
    Xf = torch.tile(image_to_explain_tensor, dims=(N*B,1,1,1))                  # 3*H*W    -> NB*3*H*W
    Xb = torch.tile(background_tensors, dims=(N,1,1,1))                         # 3*H*W    -> NB*3*H*W

    X = torch.where(masks_tensor, Xf, Xb) # T=Xf, F=Xb
    result = f(X)
    del X, Xb, Xf, masks_tensor
    result = result.reshape((-1, B, result.shape[1]))
    return np.mean(result, axis=1)

def f_masked(masks):
    global model_type
    return np.expand_dims(f_masked_resnet(masks)[:,intrested_class], axis =-1)

# print(f'masking method : {f_masked.__name__}, model_type: {model_type}')
import torch.nn.functional as F
def f(x):
    return model(x).cpu().detach().numpy()

## Load GroundTruth (Fix for CelebA-HQ dataset)

In [ ]:
low_lim = 0
anno_pth = []
for id_l, max_l in enumerate(list(range(1999,30000,2000))):
    upp_lim = max_l
    anno_pth.append((id_l, low_lim, upp_lim))
    low_lim+=2000
def find_folder(image_no, folders):
    for folder, min_no, max_no in folders:
        if min_no <= image_no <= max_no:
            return folder
    return None  # In case the image number doesn't fall into any range
# image_no = 10471
# folder = find_folder(image_no, anno_pth)
# folder

In [ ]:
ground_truth,ground_truths = None,None
union_ground_truths = []
ground_truths = []
brown_hair,black_hair,blond_hair = None,None,None
Eyeglasses = None
def load_ground_truth(intrested_class_lss,img_id=None, verbose=False):
    global ground_truth,ground_truths,union_ground_truth,union_ground_truths
    global brown_hair,Eyeglasses,anno_path

    if verbose:
        print('LAODING GT for ', intrested_class_lss)
    if att_csv_celeba:
        image_id = f'{img_id+1:06}.jpg'
    else:
        image_id = f'{img_id}.jpg'
    union_ground_truths = {}
    ground_truths = []
    for mask_1 in intrested_class_lss:
        ground_truths = []
        if mask_1 == 'Smiling':
            sub_masks = ['l_lip', 'mouth', 'u_lip']
        elif mask_1=='Eyeglasses':
            sub_masks = ['eye_g']
            Eyeglasses = df_attr[df_attr.image_id==image_id]['Eyeglasses'].values[0]==1
        #Black_Hair, Blond_Hair,Brown_Hair,Gray_Hair,Straight_Hair,Wavy_Hair
        elif mask_1=='Brown_Hair':
            brown_hair = df_attr[df_attr.image_id==image_id]['Brown_Hair'].values[0]==1
            sub_masks = ['hair']
        elif mask_1=='Black_Hair':
            black_hair = df_attr[df_attr.image_id==image_id]['Black_Hair'].values[0]==1
            sub_masks = ['hair']
        elif mask_1=='Blond_Hair':
            blond_hair = df_attr[df_attr.image_id==image_id]['Blond_Hair'].values[0]==1
            sub_masks = ['hair']
        elif mask_1=='Gray_Hair':
            brown_hair = df_attr[df_attr.image_id==image_id]['Gray_Hair'].values[0]==1
            sub_masks = ['hair']
        elif mask_1=='Straight_Hair':
            brown_hair = df_attr[df_attr.image_id==image_id]['Straight_Hair'].values[0]==1
            sub_masks = ['hair']
        elif mask_1=='Wavy_Hair':
            brown_hair = df_attr[df_attr.image_id==image_id]['Wavy_Hair'].values[0]==1
            sub_masks = ['hair']
               
        else:
            sub_masks = [mask_1]
        
        # Loop over each sub-mask
        for sub_mas in sub_masks:
            
            folder = find_folder(img_id, anno_pth)
            anno_subpath = os.path.join(ds_anno_path,str(folder))
            anno_filename = f'{os.path.join(anno_subpath, f"{img_id:05}_{sub_mas}")}.png'
            #print('sub_masks:',sub_masks,anno_filename,os.path.exists(anno_filename))
            # print(anno_filename,img_id,image_id,os.path.exists(anno_filename), brown_hair, Eyeglasses, df_attr[df_attr.image_id==image_id]['Eyeglasses'])
            #if not os.path.exists(anno_filename) or not brown_hair or not Eyeglasses and load_positives_only:
            if not os.path.exists(anno_filename):
                ground_truth = np.zeros((224,224)).astype(int)
            else:
                #print('anno_subpath: \t',anno_filename,img_id,mask_1)
                ground_truth = cv2.imread(anno_filename, cv2.IMREAD_COLOR)[:, :, ::-1]
                ground_truth = cv2.resize(ground_truth, [224, 224], interpolation=cv2.INTER_NEAREST)
                ground_truth = ground_truth[:,:,0].astype(int)
            ground_truths.append(ground_truth)
        
        # Calculate the union of the ground truths for the current mask
        if len(ground_truths) > 0:
            union_ground_truth = ground_truths[0]
            for ground_truth in ground_truths[1:]:
                union_ground_truth = cv2.bitwise_or(union_ground_truth, ground_truth)
        else:
            union_ground_truth = ground_truths[0]
        
        # Append the union ground truth to the list of all unions
        # union_ground_truths.append(union_ground_truth)
        union_ground_truths[mask_1] = union_ground_truth



        # # evaluate the ground truth mask with the background replacement strategy for masking function
        # predicted_fG = f_masked(np.expand_dims(ground_truth, axis=0))[0]
        # f_G = float(predicted_fG[predicted_cls])
        # print(class_names[predicted_cls], f_G, predicted_cls, f_G)
        # print('softmax prob:', np_softmax(predicted_fG)[predicted_cls])

        # # evaluate the backgrounf (negative of the ground truth mask)
        # background_mask = np.logical_not(ground_truth)
        # predicted_fB = f_masked(np.expand_dims(background_mask, axis=0))[0]
        # f_B = float(predicted_fB[predicted_cls])
        # print(class_names[predicted_cls], f_B, predicted_cls, f_B)
        # print('softmax prob:', np_softmax(predicted_fB)[predicted_cls])

        # print()
        # print('nu(S):  ', round(f_S, 4))
        # print('nu(G):  ', round(f_G, 4))
        # print('nu(S/G):', round(f_B, 4))
        # print('nu(0):  ', round(f_0, 4))

## Load Image

In [ ]:
def load_image(img_id=None,bg_type='blurred'):
    global image_to_explain,image_to_explain_preproc,image_to_explain_tensor,label_to_explain,background_image_set,background_tensors
    global background_image_preproc_set
    global intrested_class_lss
    img_filename = f'{os.path.join(ds_imgs_path,str(img_id))}.jpg'
    
    image_to_explain       = cv2.resize(cv2.imread(img_filename, cv2.IMREAD_COLOR),[224,224])[:,:,::-1]
    # Loop over each mask in mask_ls
    # 
    
    image_to_explain_preproc  = valid_transform(Image.fromarray(image_to_explain))
    image_to_explain_tensor = image_to_explain_preproc.to(device)

    
    bkgnd0 = np.full_like(image_to_explain, 0)
    bkgnd1 = np.full_like(image_to_explain, 127)
    bkgnd2 = np.full_like(image_to_explain, 255)
    bkgnd3 = gaussian(image_to_explain, 8, channel_axis=-1)*255
    bkgnd4 = np.clip(np.random.normal(128, 128, size=image_to_explain.shape), 0, 255).astype(np.uint8)
    bkgnd4 = (gaussian(bkgnd4, 2.0, channel_axis=-1) * 255).astype(np.uint8)
    if bg_type=='black':
        background_image_set = np.array([bkgnd0])
    elif bg_type=='gray':
        background_image_set = np.array([bkgnd1])
    elif bg_type=='white':
        background_image_set = np.array([bkgnd2])
    elif bg_type=='blurred':
        background_image_set = np.array([bkgnd3])
    elif bg_type=='noise':
        background_image_set = np.array([bkgnd4])
    elif bg_type=='full':
        background_image_set = np.array([bkgnd0, bkgnd1, bkgnd2, bkgnd3, bkgnd4])
    # if model_type=='ideal':
        # background_image_preproc_set = [torch.zeros(tuple(reversed(bkgnd.shape)))
        #                                 for bkgnd in background_image_set]
    # else:   
    # [print() for bkgnd in background_image_set]
    background_image_preproc_set = [valid_transform(Image.fromarray(bkgnd.astype(np.uint8)))
                                        for bkgnd in background_image_set]
    
    background_tensors = torch.cat([torch.unsqueeze(bk_p, dim=0) 
                                    for bk_p in background_image_preproc_set]).to(device)

## Load Image to Explain

In [ ]:
def load_image_to_explain(intrested_class,img_id=None,bg_type='blurred', verbose=False):
    global predicted_fS, predicted_f0, predicted_cls, f_S, f_0
    global model_type,pretrained_model_type
    load_image(img_id=img_id,bg_type=bg_type)
    h,w,_ = image_to_explain.shape
    # Foreground image to be explained   
    predicted_fS = f(image_to_explain_tensor.unsqueeze(0))[0]
    #sorted_classes = np.flip(np.argsort(predicted_fS))

    # predicted_cls = sorted_classes[0]
    # if 
    f_S = float(predicted_fS[intrested_class])
    if verbose:
        print(f'LOADING INTERESTED CLASS: {intrested_class}')
    # print(f'predicted_fS: {predicted_fS}')
    

    #####################
    predicted_f0 = [f(bkgnd.unsqueeze(0))[0] for bkgnd in background_tensors]
    predicted_f0 = np.mean(predicted_f0,axis=0)
    f_0          = float(predicted_f0[intrested_class])
    if verbose:
        print(f'f_S: {f_S}')
        print(f'f_0: {f_0}')

        # print(f'predicted_fS: {predicted_fS}')

## GT and IOU

In [ ]:
def combine_groundtruth_explanation(gtruth, heatmap, threshold):
    if gtruth.ndim == 3:
        gt = gtruth[:,:,0]>0
    else:
        gt = gtruth[:,:]>0
    ht = (heatmap >= threshold).astype(np.uint8)
    img = np.zeros(shape=list(heatmap.shape)+[3], dtype=np.uint8)
    img[:,:,0] = 255*(1-gt)
    img[:,:,1] = 255*(1-ht)
    img[:,:,2] = 255*(1-ht)
    return img


## Func: calc_IoU_curve

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
def calc_IoU_curve(y_true, y_pred, add_noise=True):
    
    assert isinstance(y_true, np.ndarray)
    assert isinstance(y_pred, np.ndarray)
    assert len(y_true.shape)==1 and len(y_pred.shape)==1 # assumes y_true and y_pred to be flattened arrays
    assert len(y_true)==len(y_pred)
    assert y_true.dtype==np.dtype('bool') and np.issubdtype(y_pred.dtype, np.floating)
    if add_noise:
        rng = np.random.default_rng(12345)
        y_pred = y_pred + rng.normal(0.0, 0.000000001, size=y_pred.shape)

    yd = np.array(sorted(zip(y_pred, y_true), reverse=True))
    X2   = np.zeros(len(y_pred))
    IoU2 = np.zeros(len(y_pred))
    Th   = np.zeros(len(y_pred))
    
    nT = np.sum(y_true)
    nInt = 0
    for i in range(len(y_pred)):
        if yd[i,1]: 
            nInt += 1
        
        # IoU2[i] = nInt / (i + nT - nInt)
        denom = i + nT - nInt
        IoU2[i] = nInt / denom if denom != 0 else 0.0  # or np.nan
        X2[i] = i
        Th[i] = yd[i,0]
        
    X2 = X2 / len(y_pred)
    auc_IoU = 0
    for i in range(1, len(y_pred)):
        auc_IoU += (X2[i] - X2[i-1]) * (IoU2[i] + IoU2[i-1]) / 2.0
    
    best_pt = np.argmax(IoU2)
    
    if np.sum(y_pred) == 0:
        return X2, np.zeros_like(X2), Th[best_pt], X2[best_pt], 0
    else:
        return X2, IoU2, Th[best_pt], X2[best_pt], auc_IoU

## vis_IoU

In [ ]:
def vis_IoU(shapley_values, threshold, ground_truth,verbose=False):
    
    pred = shapley_values.flatten() >= threshold
    real = ground_truth.flatten()
    image = np.full((len(pred), 3), 1.0, dtype=np.float32)
    if verbose:
        print(np.sum(pred), np.sum(real))
    image[ pred & real, : ]    = (0.0, 0.0, 0.75) # True Positives
    image[ pred & (~real), : ] = (1.0, 0.6, 0.2)  # False Positives
    image[ (~pred) & real, : ] = (1.0, 0.4, 1.0)  # False Negatives
    # plt.show()
    # plt.imshow(image.reshape(ground_truth.shape + (3,))
    #            )
    # plt.show()
    # print(np.min(shapley_values), np.min(image), np.max(image))
    # print('IoU:', np.sum(pred), np.sum(real), np.sum(pred & real), np.sum(pred & (~real)), np.sum((~pred) & real))
    return image.reshape(list(ground_truth.shape) + [3])

# XAI Methods Definitions

## Shap AA

In [ ]:
import shap_bpt
def get_aa_heatmaps(num_samples=100,verbose=False):
    explainer = shap_bpt.Explainer(f_masked, image_to_explain, 
                                   num_explained_classes=1,
                                #    explained_class_ids=[intrested_class],
                                   
                                   verbose=verbose)
    shap_values_aa = explainer.explain_instance(num_samples, method='AA', verbose_plot=False, 
                                                batch_size=batch_size)
    return shap_values_aa

## Shap BPT

In [ ]:
def get_bpt_heatmaps(num_samples=100,verbose=False):
    bpt = shap_bpt.build_bpt_from_image((image_to_explain))
    
    explainer = shap_bpt.Explainer(f_masked, image_to_explain,
                                   num_explained_classes=1,
                                    
                                #    explained_class_ids=[intrested_class],
                                     verbose=verbose)
    # print(explainer.base_f_S-explainer.base_f_0)
    shap_values_bpt = explainer.explain_instance(num_samples, method='BPT',
                                             batch_size=batch_size,bpt=bpt)
    return shap_values_bpt

## gradshap_captum

In [ ]:
# https://github.com/jacobgil/pytorch-grad-cam
from captum.attr import GradientShap
def get_gradshap_captum_multi(n_samples=50):
    global predicted_fS, predicted_f0
    global num_explained_classes
    img_4d = image_to_explain_tensor.unsqueeze(0)
    rand_img_dist = torch.cat([img_4d * 0, img_4d * 1])
    
    gradient_shap = GradientShap(model)
    heatmaps = []
    for clsid in sorted_classes[:num_explained_classes]:
        clsid = torch.tensor(clsid)
        heatmap = gradient_shap.attribute(img_4d,
                                              n_samples=n_samples,
                                              stdevs=0.0001,
                                              baselines=rand_img_dist,
                                              target=intrested_class)
        heatmap = np.sum(heatmap.squeeze().cpu().detach().numpy(), axis=0)
        
        # heatmap = np.abs(heatmap)
        heatmap = heatmap * (predicted_fS[clsid] - predicted_f0[clsid]) / np.sum(heatmap)
        heatmaps.append(heatmap)
    return np.array(heatmaps)

## LIME

In [ ]:
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries
def format_lime_heatmaps(segments, expl):
    global predicted_fS, predicted_f0
    class_heatmaps = []
    for clsid in expl.top_labels:
        heatmap = np.zeros_like(segments, dtype=np.float32)
        for segm, importance in expl.local_exp[intrested_class]:
            heatmap[ segments==segm ] += importance 
        heatmap = heatmap * (predicted_fS[clsid] - predicted_f0[clsid]) / np.sum(heatmap)
        class_heatmaps.append(heatmap)
    return np.array(class_heatmaps)
def get_segment_number(image, md):
    segmentation_fn = SegmentationAlgorithm('quickshift', kernel_size=4, max_dist=md, ratio=0.2, random_seed=1234) 
    segments = segmentation_fn(image)
    return len(np.unique(segments))

def search_segment_number(image, target_seg_no, init_max_dist=100):
    lmd, rmd = 0, init_max_dist
    lsn = get_segment_number(image, lmd)
    rsn = get_segment_number(image, rmd)
    niter = 0
    while niter<40 and rsn!=target_seg_no:
        niter += 1
        mmd = (lmd + rmd) / 2.0
        msn = get_segment_number(image, mmd)
        if msn <= target_seg_no <= lsn:
            rsn, rmd = msn, mmd
        else:
            lsn, lmd = msn, mmd
    return rmd
def lime_predict(img):
    global model_type
    return f(torch.Tensor(img).permute(0,3,1,2).to(device))
    
def get_lime_heatmaps(num_segments=100, num_samples=1000, use_stratification=False,verbose=False):
    segmentation_fn = SegmentationAlgorithm('quickshift', kernel_size=4, 
                                            max_dist=search_segment_number(image_to_explain, num_segments), 
                                            ratio=0.2, random_seed=1234) 
                                            
    segments = segmentation_fn(image_to_explain)
    def segments_getter(img):
        return segments
    num_segments = len(np.unique(segments))
    heatmap_list = []
    for bg_c in background_image_preproc_set:
        lime_explainer = lime_image.LimeImageExplainer(random_state=1234)
        lime_expl = lime_explainer.explain_instance(image_to_explain_preproc.permute(1,2,0).detach().numpy(), 
                                                    lime_predict,
                                                    top_labels=40,
                                                    # use_stratification=use_stratification,
                                                    segmentation_fn=segments_getter,
                                                    hide_color=bg_c.permute(1,2,0).detach().numpy(), 
                                                    num_samples=num_samples,
                                                    progress_bar=verbose)
        # Extract explanation for the specific class
        # lime_expl = lime_expl.local_exp[intrested_class]

        if isinstance(lime_expl, tuple):
            lime_expl = lime_expl[2]
        heatmap_list.append(format_lime_heatmaps(segments, lime_expl))
    return np.mean(heatmap_list, axis=0)

## LRP

In [ ]:
# LRP function
from captum.attr import LRP
def get_LRP_captum_heatmaps(compute=False):
    lrp = LRP(model)
    if not compute:
        return np.zeros((1,image_to_explain.shape[0],image_to_explain.shape[1]))
    heatmaps = []
    torch.cuda.empty_cache()
    for clsid in sorted_classes[:num_explained_classes]:
        clsid = torch.tensor(clsid).to(device)
        # print(clsid,class_names[clsid])
        heatmap = lrp.attribute(image_to_explain_tensor.unsqueeze(0),
                                target=clsid)
        heatmap = heatmap.squeeze().cpu().detach().numpy()
        heatmap = np.mean(heatmap, axis=0)
        heatmap = np.abs(heatmap)
        heatmap = heatmap * (predicted_fS[clsid] - predicted_f0[clsid]) / np.sum(heatmap)
        heatmaps.append(heatmap)
        del heatmap
    torch.cuda.empty_cache()
    del lrp,clsid
    return np.array(heatmaps)

## GradCAM

In [ ]:
# https://github.com/jacobgil/pytorch-grad-cam
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

target_layers = [model.unit8.conv]

def get_gradcam_heatmaps():
    global predicted_fS, predicted_f0
    
    cam = GradCAM(model=model, target_layers=target_layers)
    
    heatmaps = []
    for clsid in sorted_classes[:intrested_class]:
        heatmap = cam(input_tensor=torch.unsqueeze(image_to_explain_tensor, dim=0), 
                      targets=[ClassifierOutputTarget(clsid)])[0]
        # Normalize the heatmap
        heatmap = heatmap * (predicted_fS[clsid] - predicted_f0[clsid]) / np.sum(heatmap)
        heatmaps.append(heatmap)
    return np.array(heatmaps)

In [ ]:
# heatmap = get_gradcam_heatmaps()[0]
# heatmap

## Grad-Shap

In [ ]:
# https://github.com/jacobgil/pytorch-grad-cam
# https://captum.ai/api/gradient_shap.html
from captum.attr import GradientShap
def get_gradshap_captum_heatmaps(n_samples=50,use_abs=False):

    torch.cuda.empty_cache()
    #rand_img_dist = torch.cat([image_to_explain_tensor.unsqueeze(0) * 0, image_to_explain_tensor.unsqueeze(0) * 1])
    
    #print('rand_img_dist:', rand_img_dist.shape)
    gradient_shap = GradientShap(model)
    
    heatmaps = []
    for clsid in sorted_classes[:num_explained_classes]:
        clsid = torch.tensor(clsid)
        
        heatmap = gradient_shap.attribute(image_to_explain_tensor.unsqueeze(0),
                                          n_samples=n_samples,
                                              stdevs=0.0001,
                                              baselines=background_tensors,
                                              target=clsid)
        heatmap = np.sum(heatmap.squeeze().cpu().detach().numpy(), axis=0)
        if use_abs:
            heatmap = np.abs(heatmap)
        heatmap = heatmap * (predicted_fS[clsid] - predicted_f0[clsid]) / np.sum(heatmap)
        heatmaps.append(heatmap)
        del heatmap
    torch.cuda.empty_cache()
    del gradient_shap,clsid
    #del rand_img_dist
    return np.array(heatmaps)

## Integrated Decision Gradients (IDG)
 - https://github.com/chasewalker26/Integrated-Decision-Gradients/tree/main/util/attribution_methods

In [ ]:
# https://arxiv.org/pdf/2305.20052.pdf
# https://github.com/chasewalker26/Integrated-Decision-Gradients/tree/main
#sys.path.append(f'{path_repos}/Integrated-Decision-Gradients/util/attribution_methods')
sys.path.append(f'{path_repos}')
from saliencyMethods import IDG
def get_idg_heatmaps(use_abs=False):
    global background_tensors
    # global predicted_fS, predicted_f0
    steps = 50
    batch_si = 25 # default 25
    if background_tensors.shape[0]>1:
        background_tensors_rt=background_tensors[0:1, :, :, :]
    else:
        background_tensors_rt = background_tensors
    baseline = background_tensors_rt
    heatmaps = []
    torch.cuda.empty_cache()
    for clsid in sorted_classes[:num_explained_classes]:
        
        heatmap = idg = IDG(torch.unsqueeze(image_to_explain_tensor, dim=0), model, 
                            steps, batch_si, baseline, device, clsid)
        heatmap = idg.detach().cpu().numpy()
        heatmap = np.mean(heatmap, axis=0) # reduce to one attribution per pixel
        # normalize
        if use_abs:
            heatmap = np.abs(heatmap)
        # heatmap -= np.min(heatmap)
        heatmap = heatmap * (predicted_fS[clsid] - predicted_f0[clsid]) / np.sum(heatmap)
        heatmaps.append(heatmap)
        del heatmap
    torch.cuda.empty_cache()
    del steps,baseline,batch_si
    return np.array(heatmaps)

## GradientExplainer SHAP
 - https://shap-lrjball.readthedocs.io/en/latest/generated/shap.GradientExplainer.html

In [ ]:
import shap
def get_gradexpl_heatmap(use_abs=False):
    global background_tensors
    torch.cuda.empty_cache()
    if background_tensors.shape[0]>1:
        background_tensors_rt=background_tensors[0:1, :, :, :]
        
    else:
        background_tensors_rt = background_tensors
    baseline = background_tensors_rt
    
    e = shap.GradientExplainer(model, baseline)
    expl = e.shap_values(torch.unsqueeze(image_to_explain_tensor, dim=0),
                         nsamples=20, 
#                         ranked_outputs=len(intrested_class_lss),
                         output_rank_order='custom', # “max”, “min”, “max_abs”,
                         rseed=1234,
                        )
    heatmaps = []

    for clsid in sorted_classes[:num_explained_classes]:
        heatmap = np.sum(expl[0][:,:,:,clsid], axis=0)
        if use_abs:
            heatmap = np.abs(heatmap)
        heatmap = heatmap * (predicted_fS[clsid] - predicted_f0[clsid]) / np.sum(heatmap)
        heatmaps.append(heatmap)
    torch.cuda.empty_cache()
    del e, expl
    return np.array(heatmaps)

In [ ]:
# heatmap = get_gradexpl_heatmap(use_abs=True)
# plt.imshow(image_to_explain); plt.show()
# plt.imshow(heatmap[0])


# Combine all XAI Methods

In [ ]:
#     name,                 color,                  functor
verbose = False
methods = [ 
    ('BPT-100',         'xkcd:light pink',     partial(get_bpt_heatmaps, num_samples=100,verbose=verbose)),
    ('BPT-500',         'xkcd:bright pink',     partial(get_bpt_heatmaps, num_samples=500,verbose=verbose)),
    ('BPT-1000',         'xkcd:deep pink',     partial(get_bpt_heatmaps, num_samples=1000,verbose=verbose)),
    ]

methods_aa = [ # if multiple backgrounds
    ('AA-100', 'xkcd:bright blue',     partial(get_aa_heatmaps, num_samples=100, verbose=verbose)),
    ('AA-500', 'xkcd:bright blue',     partial(get_aa_heatmaps, num_samples=500, verbose=verbose)),
    ('AA-1000', 'xkcd:bright blue',    partial(get_aa_heatmaps, num_samples=1000, verbose=verbose)),
]
methods_lime = [
    ('LIME-100',        'xkcd:bright lime',     partial(get_lime_heatmaps, num_segments=20, num_samples=100,verbose=verbose)),
    ('LIME-500',        'xkcd:kermit green',   partial(get_lime_heatmaps, num_segments=100, num_samples=500,verbose=verbose)),
    ('LIME-1000',        'xkcd:dark lime green',partial(get_lime_heatmaps, num_segments=200, num_samples=1000,verbose=verbose))
    ]

methods_LRP = [
    ('LRP',        'xkcd:bright lime',     get_LRP_captum_heatmaps)
    ]
methods_gradcam = [
    ('GradCAM',     'xkcd:camel',            get_gradcam_heatmaps)
    ]
methods_cam = [
    ('aIDG',         'xkcd:indigo',          partial(get_idg_heatmaps, use_abs=True)),
    ('aGradExpl',    'red',                  partial(get_gradexpl_heatmap, use_abs=True))
    ]

methods_GradShapE = [
    ('GradShapE',     'xkcd:camel',            partial(get_gradshap_captum_heatmaps, n_samples=50, use_abs=True))
]

methods += methods_aa
methods += methods_lime
methods += methods_LRP
methods += methods_gradcam
methods += methods_cam
methods += methods_GradShapE

for n,_,_ in methods:
    print(n)

methods_ls = [n for  n,_,_ in methods]
methods_ls

# Functions

## saliency_to_auc

In [ ]:
# def saliency_to_auc(heatmap, batch_size=4, method='del', num_samples=101,add_noise=True):
#     if add_noise:
#         rng = np.random.default_rng(12345)
#         heatmap = heatmap + rng.normal(0.0, 0.0000001, size=heatmap.shape)
#     xs, ys, ms, masks, qs = [], [], [], [], []
#     #heatmap = gaussian(heatmap, 2.0)
#     for i in np.linspace(start=1.0, stop=0.0, num=num_samples):
#         if method=='del':
#             epsilon = (1 if i==0.0 else 0)
#             q = (np.quantile(heatmap, q=i) - epsilon)
#             m = heatmap <= q
#             nx = (1.0 - np.sum(m) / m.size)
#         elif method=='ins':
#             epsilon = (1 if i==1.0 else 0)
#             q = (np.quantile(heatmap, q=i) + epsilon)
#             m = heatmap >= q
#             nx = (np.sum(m) / m.size)
#         else:
#             raise Exception()
            
#         if len(xs)==0 or nx != xs[-1]:
#             xs.append(nx)
#             masks.append(m)
#             ms.append(np.sum(heatmap[m]))
#             qs.append(q)
#             if len(masks) >= batch_size:
#                 y = f_masked(np.array(masks))[:, predicted_cls]
#                 ys.extend(y)
#                 masks = []

#     if len(masks) > 0:
#         y = f_masked(np.array(masks))[:, predicted_cls]
#         ys.extend(y)
    
#     xs, ys = np.array(xs), np.array(ys)
#     auc_reg, auc_eff, auc_mse = 0.0, 0.0, 0.0
#     assert(len(xs) == len(ys))
#     # compute the area under the curve - use the rectangle method
#     for i in range(1, len(xs)):
#         delta_x = abs(xs[i] - xs[i-1])
#         if delta_x > 0:
#             auc_reg += abs(delta_x * 0.5*(ys[i-1] + ys[i])) # base  height
#             auc_eff += abs(delta_x * 0.5*(ys[i-1] + ys[i] - ms[i-1] - ms[i])) # base *  height
#             auc_mse += abs(delta_x * 0.5*(ys[i-1] + ys[i] - ms[i-1] - ms[i])**2) # base  height^2
    
#     # return xs, ys, ms, auc_reg, auc_eff, auc_mse
#     return {'xs':xs, 'ys':ys, 'ms':ms, 'qs':qs, 'auc_reg':auc_reg, 'auc_eff':auc_eff, 'auc_mse':auc_mse,'method':method}

In [ ]:
# def saliency_to_auc(f_masked, heatmap, f_S, f_0, predicted_cls, batch_size=4, method='del', num_samples=101, rule='trapezoid'):
#     assert isinstance(heatmap, np.ndarray), "Heatmap must be a numpy array"
#     assert len(heatmap.shape) == 2, "Heatmap must be 2D"
#     assert np.issubdtype(heatmap.dtype, np.floating), "Heatmap must be float type"

#     nu_max = max(f_S, f_0)
#     nu_min = min(f_S, f_0)
#     heatmap = np.nan_to_num(heatmap, nan=0.0)

#     xs, ys, ms, masks, qs = [], [], [], [], []

#     for i, value in enumerate(np.linspace(1.0, 0.0, num=num_samples)):
#         if method == 'del':
#             epsilon = 1e-8 if value == 0.0 else 0.0
#             q = np.quantile(heatmap, q=value) - epsilon
#             m = heatmap <= q
#             nx = 1.0 - np.sum(m) / m.size
#         elif method == 'ins':
#             epsilon = 1e-8 if value == 1.0 else 0.0
#             q = np.quantile(heatmap, q=value) + epsilon
#             m = heatmap >= q
#             nx = np.sum(m) / m.size
#         else:
#             raise ValueError("Unsupported method. Use 'del' or 'ins'.")
        
#         print(i, value, q, nx)

#         if len(xs) == 0 or nx != xs[-1]:
#             assert m.dtype == bool and len(m.shape) == 2
#             xs.append(nx)
#             masks.append(m)
#             ms.append(np.sum(heatmap[m]))
#             qs.append(q)
#         # if predicted_cls >= f_masked(np.array(masks)).shape[1]:
#         #     predicted_cls = 0  # fallback for CelebA
#         # y = f_masked(np.array(masks))[:, predicted_cls]

#         outputs = f_masked(np.array(masks))
#         if predicted_cls >= outputs.shape[1]:
#             predicted_cls = 0
#         if len(masks) >= batch_size or (len(masks) > 0 and i == (num_samples - 1)):
#             y = f_masked(np.array(masks))[:, predicted_cls]
#             ys.extend(y)
#             masks = []

#     xs, ys = np.array(xs), np.array(ys)
#     y_adjusted = np.clip(ys, nu_min, nu_max)
#     y_adjusted_rescaled = (y_adjusted - nu_min) / (nu_max - nu_min + 1e-8)
#     ys_rescaled = (ys - nu_min) / (nu_max - nu_min + 1e-8)

#     auc, auc_r, auc_mae, auc_mse, auc_adj, auc_adjr = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
#     curve_range = range(1, len(xs)) if rule == 'trapezoid' else range(len(xs))

#     for i in curve_range:
#         if rule == 'trapezoid':
#             delta_x = abs(xs[i] - xs[i - 1])
#             y_mid = 0.5 * (ys[i - 1] + ys[i])
#             y_r_mid = 0.5 * (ys_rescaled[i - 1] + ys_rescaled[i])
#             err_mid = y_mid - 0.5 * (ms[i - 1] - ms[i])
#             y_adj_mid = 0.5 * (y_adjusted[i - 1] + y_adjusted[i])
#             y_adjr_mid = 0.5 * (y_adjusted_rescaled[i - 1] + y_adjusted_rescaled[i])
#         else:
#             delta_x = 1.0 / num_samples if i == len(xs) - 1 else abs(xs[i + 1] - xs[i])
#             y_mid = ys[i]
#             y_r_mid = ys_rescaled[i]
#             err_mid = y_mid - ms[i]
#             y_adj_mid = y_adjusted[i]
#             y_adjr_mid = y_adjusted_rescaled[i]

#         auc += delta_x * y_mid
#         auc_r += delta_x * y_r_mid
#         auc_mae += delta_x * abs(err_mid)
#         auc_mse += delta_x * (err_mid ** 2)
#         auc_adj += delta_x * y_adj_mid
#         auc_adjr += delta_x * y_adjr_mid

#     return {
#         'xs': xs,
#         'ys': ys,
#         'ms': ms,
#         'qs': qs,
#         'ysr': ys_rescaled,
#         'y_adj': y_adjusted,
#         'y_adjr': y_adjusted_rescaled,
#         'method': method,
#         'auc': auc,
#         'auc_r': auc_r,
#         'auc_mae': auc_mae,
#         'auc_mse': auc_mse,
#         'auc_rmse': np.sqrt(auc_mse),
#         'auc_adj': auc_adj,
#         'auc_adjr': auc_adjr
#     }


In [ ]:
# def saliency_to_auc(f_masked,heatmap, f_S, f_0, predicted_cls, batch_size=4, method='del', num_samples=101, 
#                     rule='trapezoid'):

#     # print(f'Computing AUC with method={method}, batch_size={batch_size}, num_samples={num_samples}, rule={rule}')
#     # print('from saliency_to_auc',heatmap.shape, heatmap.dtype, heatmap.min(), heatmap.max())
#     assert isinstance(heatmap, np.ndarray)
#     assert len(heatmap.shape)==2 and np.issubdtype(heatmap.dtype, np.floating)

#     nu_max = max(f_S, f_0)
#     nu_min = min(f_S, f_0)

#     xs, ys, ms, masks, qs = [], [], [], [], []
#     for i, value in enumerate(np.linspace(start=1.0, stop=0.0, num=num_samples)):
#         if method=='del':
#             epsilon = (1 if value==0.0 else 0)
#             q = (np.quantile(heatmap, q=value) - epsilon)
#             m = heatmap <= q
#             nx = (1.0 - np.sum(m) / m.size)
#         elif method=='ins':
#             epsilon = (1 if value==1.0 else 0)
#             q = (np.quantile(heatmap, q=value) + epsilon)
#             m = heatmap >= q
#             nx = (np.sum(m) / m.size)
#         else:
#             raise Exception()
            
#         # add a new datapoint on the curve
#         if len(xs)==0 or nx != xs[-1]: 
#             assert m.dtype==bool and len(m.shape)==2
#             xs.append(nx)
#             masks.append(m)
#             ms.append(np.sum(heatmap[m]))
#             qs.append(q)

#         # evaluate the characteristic function
#         if len(masks) >= batch_size or (len(masks)>0 and i==(num_samples-1)):
#             y = f_masked(np.array(masks))[:, predicted_cls]
#             ys.extend(y)
#             masks = []

#     assert len(masks)==0    
#     xs, ys = np.array(xs), np.array(ys)
#     assert(len(xs) == len(ys))

#     # compute considering under/over shoots
#     overshoot_max = np.maximum(0, ys - nu_max) # overshoot for values exceeding the maximum
#     overshoot_min = np.maximum(0, nu_min - ys) # overshoot for values below the minimum
#     # adjust ys with the overshoot. Clip it inside the admitted range
#     y_adjusted = np.clip(ys - 2*overshoot_max + 2*overshoot_min, nu_min, nu_max)

#     # rescaling
#     ys_rescaled = (ys - nu_min) / (nu_max - nu_min)
#     y_adjusted_rescaled = (y_adjusted - nu_min) / (nu_max - nu_min)

#     auc, auc_r, auc_mae, auc_mse, auc_adj, auc_adjr = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

#     curve_range = range(1, len(xs)) if rule=='trapezoid' else range(len(xs))

#     # compute the area under the curve with the midpoint Riemann sum (i.e. the trapezoidal rule)
#     for i in curve_range:
#         if rule=='trapezoid':
#             delta_x = abs(xs[i] - xs[i-1])
#             assert delta_x > 0
#             y_mid   =         0.5*(ys[i-1] + ys[i])
#             y_r_mid =         0.5*(ys_rescaled[i-1] + ys_rescaled[i])
#             err_mid = y_mid - 0.5*(ms[i-1] - ms[i])
#             y_adj_mid =       0.5*(y_adjusted[i-1] + y_adjusted[i])
#             y_adjr_mid =      0.5*(y_adjusted_rescaled[i-1] + y_adjusted_rescaled[i])
#         else: # rectangles
#             delta_x = 1.0/num_samples if i==len(xs)-1 else abs(xs[i+1] - xs[i])
#             assert delta_x > 0
#             y_mid   =         ys[i]
#             y_r_mid =         ys_rescaled[i]
#             err_mid = y_mid - ms[i]
#             y_adj_mid =       y_adjusted[i]
#             y_adjr_mid =      y_adjusted_rescaled[i]


#         auc += abs(delta_x * y_mid) # base * height
#         auc_r += abs(delta_x * y_r_mid) # base * height
#         # auc_eff += abs(delta_x * err_mid) # base * height
#         auc_mae += abs(delta_x * err_mid) # base * height
#         auc_mse += abs(delta_x * (err_mid**2)) # base * height^2
#         auc_adj += abs(delta_x * y_adj_mid)
#         auc_adjr += abs(delta_x * y_adjr_mid)

#     return {'xs':xs, 'ys':ys, 'ms':ms, 'qs':qs, 'ysr':ys_rescaled,
#             'y_adj':y_adjusted, 'y_adjr':y_adjusted_rescaled, 
#             'method':method, #'class_id':class_id,
#             'auc':auc, 'auc_r':auc_r, #'auc_eff':auc_eff, 
#             'auc_mae':auc_mae, 'auc_mse':auc_mse, 'auc_rmse':np.sqrt(auc_mse), 
#             'auc_adj':auc_adj, 'auc_adjr':auc_adjr}

In [ ]:
def saliency_to_auc(nu, heatmap, f_S, f_0, predicted_cls, batch_size=4, method='del', num_samples=101, 
                    rule='trapezoid'):
    assert isinstance(heatmap, np.ndarray)
    assert len(heatmap.shape)==2 and np.issubdtype(heatmap.dtype, np.floating)

    # nu_max = max(f_S, f_0)
    # nu_min = min(f_S, f_0)

    xs, ys, ms, masks, qs = [], [], [], [], []
    for i, value in enumerate(np.linspace(start=1.0, stop=0.0, num=num_samples)):
        if method=='del':
            epsilon = (1 if value==0.0 else 0)
            q = (np.quantile(heatmap, q=value) - epsilon)
            m = heatmap <= q
            nx = (1.0 - np.sum(m) / m.size)
        elif method=='ins':
            epsilon = (1 if value==1.0 else 0)
            q = (np.quantile(heatmap, q=value) + epsilon)
            m = heatmap >= q
            nx = (np.sum(m) / m.size)
        else:
            raise Exception()
            
        # add a new datapoint on the curve
        if len(xs)==0 or nx != xs[-1]: 
            assert m.dtype==bool and len(m.shape)==2
            xs.append(nx)
            masks.append(m)
            ms.append(np.sum(heatmap[m]))
            qs.append(q)

        # evaluate the characteristic function
        if len(masks) >= batch_size or (len(masks)>0 and i==(num_samples-1)):
            y = nu(np.array(masks))[:, predicted_cls]
            ys.extend(y)
            masks = []

    assert len(masks)==0    
    xs, ys = np.array(xs), np.array(ys)
    assert(len(xs) == len(ys))

    # compute considering under/over shoots
    if f_S > f_0:
        overshoot_max = np.maximum(0, ys - f_S) # overshoot for values exceeding the maximum f(S)
        overshoot_min = np.maximum(0, f_0 - ys) # overshoot for values below the minimum f(0)
    else: # f(S) < f(0)
        overshoot_max = np.maximum(0, ys - f_0) # overshoot for values exceeding the maximum f(0)
        overshoot_min = np.maximum(0, f_S - ys) # overshoot for values below the minimum f(S)

    # clip ys, no oveshoots
    y_clipped = np.clip(ys, min(f_S, f_0), max(f_S, f_0))
    # adjust ys with the overshoot. Clip it inside the admitted range
    y_adjusted = np.clip(ys - 2*overshoot_max + 2*overshoot_min, min(f_S, f_0), max(f_S, f_0))

    # rebase to f(0)
    if f_S > f_0:
        flipped = False
        ys = ys - f_0 
        y_clipped = y_clipped - f_0 
        y_adjusted = y_adjusted - f_0
    else: # f(S) < f(0)
        flipped = True
        ys = f_0 - ys 
        y_clipped = f_0 - y_clipped 
        y_adjusted = f_0 - y_adjusted

    # rescaling
    ys_rescaled = ys / abs(f_S - f_0)
    y_clipped_rescaled = y_clipped / abs(f_S - f_0)
    y_adjusted_rescaled = y_adjusted / abs(f_S - f_0)

    auc, auc_r, auc_mae, auc_mse, auc_adj, auc_adjr, auc_clip, auc_clipr = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

    curve_range = range(1, len(xs)) if rule=='trapezoid' else range(len(xs))

    # compute the area under the curve with the midpoint Riemann sum (i.e. the trapezoidal rule)
    for i in curve_range:
        if rule=='trapezoid':
            delta_x = abs(xs[i] - xs[i-1])
            assert delta_x > 0
            y_mid   =         0.5*(ys[i-1] + ys[i])
            y_r_mid =         0.5*(ys_rescaled[i-1] + ys_rescaled[i])
            err_mid = y_mid - 0.5*(ms[i-1] - ms[i])
            y_clip_mid =       0.5*(y_clipped[i-1] + y_clipped[i])
            y_clipr_mid =      0.5*(y_clipped_rescaled[i-1] + y_clipped_rescaled[i])
            y_adj_mid =       0.5*(y_adjusted[i-1] + y_adjusted[i])
            y_adjr_mid =      0.5*(y_adjusted_rescaled[i-1] + y_adjusted_rescaled[i])
        else: # rectangles
            delta_x = 1.0/num_samples if i==len(xs)-1 else abs(xs[i+1] - xs[i])
            assert delta_x > 0
            y_mid   =         ys[i]
            y_r_mid =         ys_rescaled[i]
            err_mid = y_mid - ms[i]
            y_clip_mid =       y_clipped[i]
            y_clipr_mid =      y_clipped_rescaled[i]
            y_adj_mid =       y_adjusted[i]
            y_adjr_mid =      y_adjusted_rescaled[i]


        auc += abs(delta_x * y_mid) # base * height
        auc_r += abs(delta_x * y_r_mid) # base * height
        # auc_eff += abs(delta_x * err_mid) # base * height
        auc_mae += abs(delta_x * err_mid) # base * height
        auc_mse += abs(delta_x * (err_mid**2)) # base * height^2
        auc_clip += abs(delta_x * y_clip_mid)
        auc_clipr += abs(delta_x * y_clipr_mid)
        auc_adj += abs(delta_x * y_adj_mid)
        auc_adjr += abs(delta_x * y_adjr_mid)

    return {'xs':xs, 'ms':ms, 'qs':qs, 
            'f_0':f_0, 'f_S':f_S, 'flipped':flipped, 
            'ys':ys, 'ysr':ys_rescaled,
            'y_clip':y_clipped, 'y_clipr':y_clipped_rescaled, 
            'y_adj':y_adjusted, 'y_adjr':y_adjusted_rescaled, 
            'method':method, 'predicted_cls':predicted_cls,
            'auc':auc, 'auc_r':auc_r,
            'auc_mae':auc_mae, 'auc_mse':auc_mse, 'auc_rmse':np.sqrt(auc_mse), 
            'auc_clip':auc_clip, 'auc_clipr':auc_clipr,
            'auc_adj':auc_adj, 'auc_adjr':auc_adjr}

## Plotting Functions

In [ ]:
def plot_heatmaps(image_to_explain,ground_truth_final,heatmaps,image_name=None,save_dir=None,plot_title=False,bg_type=None,annotate_text=False,
                  save_figs=False, ext='png',dpi=50,fontsize=22,destroy_fig=False):
    
    fig,axes = plt.subplots(1, len(methods)+2, figsize=(2*(len(methods)+2), 3))
    axes[0].imshow(image_to_explain)
    axes[1].imshow(ground_truth_final, cmap='binary')
    cn,ic,pfS,pf0 = classes_names,intrested_class,predicted_fS,predicted_f0
    for n_id, (n,_,_) in tqdm(enumerate(methods), desc='IoU',leave=False):
        
        ax = axes[n_id+2]
        vmax = np.quantile(np.abs(heatmaps[n][0]), 0.99)
        h = heatmaps[n][0]
        ax.imshow(h, cmap=shap_bpt.shapley_values_colormap, vmin=-vmax, vmax=vmax)
    if annotate_text:
        axes[0].text(0.05, image_to_explain.shape[0]+30, 
                    f'{ic} : {cn[ic]} : '+ r'$\nu(N) - \nu(\varnothing)$'+f' = [{pfS[ic]:0.6}]-[{pf0[ic]:0.6}]={pfS[ic]-pf0[ic]:0.6}',
                    fontsize=18)
    if plot_title:
        methods_names = [n for n,_,_ in methods]
        methods_names.insert(0, 'Input')
        methods_names.insert(1, 'Gr Truth')
        for (ax,ttl) in zip(axes,methods_names):
            ax.set_title(ttl, fontsize=fontsize);
    for ax in axes:
        ax.set_xticks([]); ax.set_yticks([])
    plt.subplots_adjust(hspace=0.05,wspace=0.05)
    #plt.tight_layout(pad = 0.05)
    if save_dir is None:
        save_dir = results_selected
    if save_figs:
        if bg_type is None:
            bg_type = background_type
        plt.savefig(f'{save_dir}/{image_name}_{classes_names[intrested_class]}_{bg_type}_heatmaps.{ext}', dpi=dpi, bbox_inches='tight', pad_inches=0.02)
    if destroy_fig:
        plt.close()
    plt.show()
    

### PARTIAL PLOTS

In [ ]:
def plot_filtered_heatmaps(heatmaps,ground_truth_final,invert_heatmap,invert_heatmap_global,image_name=None, save_dir=None, plot_title=False, bg_type=None, annotate_text=False, 
                           save_figs=False, ext='png', dpi=50, fontsize=22, destroy_fig=False, filter_methods=None,draw_gt_contour=True):
    
    # Filter methods based on user selection
    selected_methods = [(n, c, v) for n, c, v in methods if filter_methods is None or n in filter_methods]
    num_methods = len(selected_methods)

    # If no methods are selected, exit
    if num_methods == 0:
        print("No methods selected for filtering. Exiting.")
        return
    
    # Total subplots: input + ground truth + filtered methods
    num_subplots = num_methods + 2
    fig, axes = plt.subplots(1, num_subplots, figsize=(2 * num_subplots, 3))

    # Plot Input Image
    axes[0].imshow(image_to_explain)
    axes[0].set_xticks([]) 
    axes[0].set_yticks([])
    
    # Plot Ground Truth
    axes[1].imshow(ground_truth_final, cmap='binary')
    axes[1].set_xticks([]) 
    axes[1].set_yticks([])

    # Plot Heatmaps for Selected Methods
    for i, (n, _, _) in enumerate(selected_methods):
        ax = axes[i + 2]
        vmax = np.quantile(np.abs(heatmaps[n][0]), 0.99)
        
        if invert_heatmap and n in invert_heatmap_global:
            h = - heatmaps[n][0]
        else:
            h = heatmaps[n][0]
        if n in invert_heatmap_global:
            # print('abs values for ',n)
            h = np.abs(heatmaps[n][0])
        ax.imshow(h, cmap=shap_bpt.shapley_values_colormap, vmin=-vmax, vmax=vmax)
        # Draw Ground Truth Contours if enabled
        if draw_gt_contour:
            marked_h = mark_boundaries(
                np.tile((255, 255, 255, 0), (heatmaps[n][0].shape[0], heatmaps[n][0].shape[1], 1)), 
                ground_truth_final, mode='thick', color=(0, 0, 0, 1)
            )
            ax.imshow(marked_h)
    # Annotate text
    if annotate_text:
        axes[0].text(
            0.05, image_to_explain.shape[0] + 30, 
            f'{intrested_class} : {classes_names[intrested_class]} : ' + 
            r'$\nu(N) - \nu(\varnothing)$' + 
            f' = [{predicted_fS[intrested_class]:0.6}]-[{predicted_f0[intrested_class]:0.6}]={predicted_fS[intrested_class]-predicted_f0[intrested_class]:0.6}', 
            fontsize=18
        )
    
    # Set plot titles if needed
    if plot_title:
        methods_names = [n for n, _, _ in selected_methods]
        methods_names.insert(0, "Input")
        methods_names.insert(1, "Gr Truth")
        for ax, ttl in zip(axes, methods_names):
            ax.set_title(ttl, fontsize=fontsize)
    
    for ax in axes:
        ax.set_xticks([]) 
        ax.set_yticks([])

    plt.subplots_adjust(hspace=0.05, wspace=0.05)

    # Save figure if required
    if save_figs:
        if bg_type is None:
            bg_type = background_type
        if save_dir is None:
            save_dir = results_selected  # Default save directory
        
        filename = f'{save_dir}/{image_name}_{classes_names[intrested_class]}_{bg_type}_filtered_heatmaps.{ext}'
        plt.savefig(filename, dpi=dpi, bbox_inches='tight', pad_inches=0.02)

    if destroy_fig:
        plt.close()
    
    plt.show()


In [ ]:
props = dict(boxstyle='round', facecolor='white', alpha=0.6, pad=0.2)
def plot_IoU(image_name=None,save_dir=None,plot_title=False,invert_heatmap=False,bg_type=None,save_figs=False, ext='png', dpi=50,fontsize=22,destroy_fig=False):
    
    fig,axes = plt.subplots(1, len(methods)+2, figsize=(2*(len(methods)+2), 3))
    axes[0].imshow(image_to_explain)
    axes[1].imshow(ground_truth_final, cmap='binary')
    
    for n_id, (n,_,_) in tqdm(enumerate(methods), desc='IoU',leave=False):
        ax = axes[n_id+2]
        if invert_heatmap:
            heatmaps[n][0] = -heatmaps[n][0]
        img, max_IoU = vis_IoU(heatmaps[n][0], IoU[n][2], ground_truth_final.astype(bool)), np.max(IoU[n][1])
        ax.imshow(img)
        ax.text(10,45, f'IoU:{max_IoU:.3}', fontsize=16, bbox=props,weight='bold')
    if plot_title:
        methods_names = [n for n,_,_ in methods]
        methods_names.insert(0, 'Input')
        methods_names.insert(1, 'Gr Truth')
        for (ax,ttl) in zip(axes,methods_names):
            ax.set_title(ttl, fontsize=fontsize);
    for ax in axes:
        ax.set_xticks([]); ax.set_yticks([])
    plt.subplots_adjust(hspace=0.05,wspace=0.05)
    # plt.tight_layout(pad = 0.05)
    if save_dir is None:
        save_dir = results_selected
    if save_figs:
        if bg_type is None:
            bg_type = background_type
        plt.savefig(f'{save_dir}/{image_name}_{classes_names[intrested_class]}_{bg_type}_iou.{ext}', dpi=dpi, bbox_inches='tight', pad_inches=0.02)
    if destroy_fig:
        plt.close()
    plt.show()

In [ ]:
props = dict(boxstyle='round', facecolor='white', alpha=0.6, pad=0.2)

def plot_filtered_IoU(heatmaps,ground_truth_final,invert_heatmap,invert_heatmap_global,IoU,image_name=None, save_dir=None, plot_title=False, 
                       bg_type=None, save_figs=False, ext='png', dpi=50, fontsize=22, 
                       destroy_fig=False, filter_methods=None,draw_gt_contour=True):
    
    # Filter methods based on user selection
    selected_methods = [(n, c, v) for n, c, v in methods if filter_methods is None or n in filter_methods]
    num_methods = len(selected_methods)

    # If no methods are selected, exit
    if num_methods == 0:
        print("No methods selected for filtering. Exiting.")
        return
    
    # Total subplots: input + ground truth + filtered methods
    num_subplots = num_methods + 2
    fig, axes = plt.subplots(1, num_subplots, figsize=(2 * num_subplots, 3))

    # Plot Input Image
    axes[0].imshow(image_to_explain)
    axes[0].set_xticks([]) 
    axes[0].set_yticks([])

    # Plot Ground Truth
    axes[1].imshow(ground_truth_final, cmap='binary')
    axes[1].set_xticks([]) 
    axes[1].set_yticks([])

    # Plot IoU for Selected Methods
    for i, (n, _, _) in enumerate(selected_methods):
        ax = axes[i + 2]
        if invert_heatmap and n in invert_heatmap_global:
            heatmaps[n][0] = - heatmaps[n][0]
        else:
            heatmaps[n][0] = heatmaps[n][0]
        if n in invert_heatmap_global:
            # print('abs values for ',n)
            heatmaps[n][0] = np.abs(heatmaps[n][0])
        # if invert_heatmap:
        #     heatmaps[n][0] = -heatmaps[n][0]  # Invert the heatmap

        img, max_IoU = vis_IoU(heatmaps[n][0], IoU[n][2], ground_truth_final.astype(bool)), np.max(IoU[n][1])
        ax.imshow(img)
        if draw_gt_contour:
            marked_h = mark_boundaries(
                np.tile((255, 255, 255, 0), (heatmaps[n][0].shape[0], heatmaps[n][0].shape[1], 1)), 
                ground_truth_final, mode='thick', color=(0, 0, 0, 1)
            )
            ax.imshow(marked_h)

        ax.text(10, 45, f'IoU:{max_IoU:.3}', fontsize=16, bbox=props, weight='bold')

    # Set plot titles if needed
    if plot_title:
        methods_names = [n for n, _, _ in selected_methods]
        methods_names.insert(0, "Input")
        methods_names.insert(1, "Gr Truth")
        for ax, ttl in zip(axes, methods_names):
            ax.set_title(ttl, fontsize=fontsize)
    
    for ax in axes:
        ax.set_xticks([]) 
        ax.set_yticks([])

    plt.subplots_adjust(hspace=0.05, wspace=0.05)

    # Save figure if required
    if save_figs:
        if bg_type is None:
            bg_type = background_type
        if save_dir is None:
            save_dir = results_selected  # Default save directory
        
        filename = f'{save_dir}/{image_name}_{classes_names[intrested_class]}_{bg_type}_filtered_iou.{ext}'
        plt.savefig(filename, dpi=dpi, bbox_inches='tight', pad_inches=0.02)

    if destroy_fig:
        plt.close()
    
    plt.show()


In [ ]:
def plot_heatmaps_svg(image_name=None,save_dir=None,plot_title=False,bg_type=None,annotate_text=False,save_figs=False, ext='png',dpi=100,fontsize=22,destroy_fig=False):
    
    fig,axes = plt.subplots(1, len(methods)+1, figsize=(2*(len(methods)+2), 3))
    axes[0].imshow(image_to_explain)
    axes[1].imshow(ground_truth_final, cmap='binary')
    cn,ic,pfS,pf0 = classes_names, intrested_class,predicted_fS,predicted_f0
    for n_id, (n,_,_) in tqdm(enumerate(methods), desc='IoU',leave=False):
        ax = axes[n_id+1]
        vmax = np.quantile(np.abs(heatmaps[n][0]), 0.99)
        h = heatmaps[n][0]
        ax.imshow(h, cmap=shap_bpt.shapley_values_colormap, vmin=-vmax, vmax=vmax)
    if annotate_text:
        axes[0].text(0.05, image_to_explain.shape[0]+30, 
                    f'{ic} : {cn[ic]} : '+ r'$\nu(N) - \nu(\varnothing)$'+f' = {pfS[ic]-pf0[ic]:0.6}',
                    fontsize=18)
    if plot_title:
        methods_names = [n for n,_,_ in methods]
        methods_names.insert(0, 'Input')
        for (ax,ttl) in zip(axes,methods_names):
            ax.set_title(ttl, fontsize=fontsize);
    for ax in axes:
        ax.set_xticks([]); ax.set_yticks([])
    plt.subplots_adjust(hspace=0.05,wspace=0.05)
    #plt.tight_layout(pad = 0.05)
    if save_dir is None:
        save_dir = results_selected
    if save_figs:
        if bg_type is None:
            bg_type = background_type
        plt.savefig(f'{save_dir}/{image_name}_{classes_names[intrested_class]}_{bg_type}_heatmaps.{ext}', dpi=dpi, bbox_inches='tight', pad_inches=0.02)
    if destroy_fig:
        plt.close()
    plt.show()
    

In [ ]:
props = dict(boxstyle='round', facecolor='white', alpha=0.6, pad=0.2)
def plot_IoU_svg(image_name=None,save_dir=None,plot_title=False,invert_heatmap=False,bg_type=None,save_figs=False, ext='png', dpi=100,fontsize=22,destroy_fig=False):
    
    fig,axes = plt.subplots(1, len(methods)+1, figsize=(2*(len(methods)+2), 3))
    axes[0].imshow(ground_truth_final, cmap='binary')
    
    for n_id, (n,_,_) in tqdm(enumerate(methods), desc='IoU',leave=False):
        ax = axes[n_id+1]
        if invert_heatmap:
            heatmaps[n][0] = -heatmaps[n][0]
        
        img, max_IoU = vis_IoU(heatmaps[n][0], IoU[n][2], ground_truth_final.astype(bool)), np.max(IoU[n][1])
        ax.imshow(img)
        ax.text(10,45, f'IoU:{max_IoU:.3}', fontsize=16, bbox=props,weight='bold')
    if plot_title:
        methods_names = [n for n,_,_ in methods]
        methods_names.insert(1, 'Gr Truth')
        for (ax,ttl) in zip(axes,methods_names):
            ax.set_title(ttl, fontsize=fontsize);
    for ax in axes:
        ax.set_xticks([]); ax.set_yticks([])
    plt.subplots_adjust(hspace=0.05,wspace=0.05)
    # plt.tight_layout(pad = 0.05)
    if save_dir is None:
        save_dir = results_selected
    if save_figs:
        if bg_type is None:
            bg_type = background_type
        plt.savefig(f'{save_dir}/{image_name}_{classes_names[intrested_class]}_{bg_type}_iou.{ext}', dpi=dpi, bbox_inches='tight', pad_inches=0.02)
    if destroy_fig:
        plt.close()
    plt.show()

## Plotting functions to save Figures by Sub-parts

In [ ]:
def plot_heatmaps_by_parts(save_dir=None,destroy_fig=True,dpi=100,ext='svg'):
    plt.figure(figsize=(3,3))
    vmax = np.quantile(np.abs(heatmaps[n][0]), 0.99)
    h = heatmaps[n][0]
    plt.imshow(h, cmap=shap_bpt.shapley_values_colormap, vmin=-vmax, vmax=vmax)
    plt.xticks([]) ; plt.yticks([])
    plt.subplots_adjust(hspace=0.05, wspace=0.05)
    plt.savefig(f'{save_dir}/{img_id}_{classes_names[intrested_class]}_{n}.{ext}',dpi=100, bbox_inches='tight', pad_inches=0.02)
    if destroy_fig:
        plt.close()
    plt.show()

    
def plot_iou_by_parts(save_dir=None,destroy_fig=True,dpi=100,ext='svg'):
    plt.figure(figsize=(3,3))

    h = heatmaps[n][0]
    img, max_IoU = vis_IoU(heatmaps[n][0], IoU[n][2], ground_truth_final.astype(bool)), np.max(IoU[n][1])
    plt.imshow(img)
    plt.text(10,45, f'IoU:{max_IoU:.3}', fontsize=22, bbox=props,weight='bold')
    plt.xticks([]) ; plt.yticks([])
    plt.subplots_adjust(hspace=0.05, wspace=0.05)
    plt.savefig(f'{save_dir}/{img_id}_{classes_names[intrested_class]}_{n}_IoU.{ext}',dpi=100, bbox_inches='tight', pad_inches=0.02)
    if destroy_fig:
        plt.close()
    plt.show()
def plot_img_by_parts(save_dir=None,destroy_fig=True,dpi=100,ext='svg'):
    for img_idd,(img,lbl) in enumerate(zip([image_to_explain,ground_truth_final], ['img',f'gt_{classes_names[intrested_class]}'])):
        plt.figure(figsize=(3,3))
        if img_idd==0:
            plt.imshow(img)
        else: 
            plt.imshow(img, cmap = 'binary')
        plt.xticks([]) ; plt.yticks([])
        plt.subplots_adjust(hspace=0.05, wspace=0.05)
        plt.savefig(f'{save_dir}/{img_id}_{lbl}.{ext}',dpi=100, bbox_inches='tight', pad_inches=0.02)
        if destroy_fig:
            plt.close()
        plt.show()

## Select Class to Explain

In [ ]:
#Black_Hair, Blond_Hair,Brown_Hair,Gray_Hair,Straight_Hair,Wavy_Hair
#intrested_class_lss = ['Brown_Hair','Black_Hair','Gray_Hair']
intrested_class_lss = ['Brown_Hair','Eyeglasses']
#intrested_class_lss = ['Gray_Hair']

intrested_class_ls=[]
for icl in intrested_class_lss:
    intrested_class_ls.append(classes_names.index(icl))

print(intrested_class_ls, intrested_class_lss)

## Select Annotation for Selected Classes

In [ ]:
def natural_key(s):
    return [int(text) if text.isdigit() else text for text in re.split(r'(\d+)', s)]

## Filter Annotation Dataframes for Selected Classes
                            

In [ ]:
# intrested_class_lss = ['Brown_Hair']
# intrested_class_lss = ['Eyeglasses']

intrested_class_lss = ['Brown_Hair','Eyeglasses']
#intrested_class_lss = ['Brown_Hair','Eyeglasses']

intrested_class_ls=[]
for icl in intrested_class_lss:
    intrested_class_ls.append(classes_names.index(icl))

In [ ]:
# files = np.unique(df_attr_sel.image_id)
# iterations = len(files)
# sorted_filenames = sorted(files, key=natural_key)

df_attr_hairs = df_attr[['image_id','Brown_Hair','Black_Hair','Blond_Hair','Gray_Hair']].copy()

df_attr_brownhairs_true = df_attr_hairs[df_attr_hairs.Brown_Hair==1]
df_attr_brownhairs_false = df_attr_hairs[(df_attr_hairs.Brown_Hair == 0) & ((df_attr_hairs.Black_Hair == 1) | (df_attr_hairs.Blond_Hair == 1) | (df_attr_hairs.Gray_Hair == 1))]

df_attr_brownhairs = pd.concat([df_attr_brownhairs_false, df_attr_brownhairs_true]).drop_duplicates().reset_index(drop=True)

df_attr_brownhairs = df_attr_brownhairs.sort_values(by='image_id', key=lambda col: col.map(natural_key)).reset_index(drop=True)

#list_brownhairs_true = list(df_attr_brownhairs_true.image_id.values)
#list_brownhairs_false = list(df_attr_brownhairs_false.image_id.values)
list_brownhairs = list(df_attr_brownhairs.image_id.values)

print('Brown:',len(list_brownhairs),'~ Brown:',len(list_brownhairs))

In [ ]:
df_attr_brownhairs.head(10)

In [ ]:
df_attr_eyeglasses = df_attr[['image_id','Eyeglasses']].copy()
df_attr_eyeglasses_true = df_attr_eyeglasses[df_attr_eyeglasses.Eyeglasses==1]

list_eyeglasses_true = list(df_attr_eyeglasses_true.image_id.values)

print('Eyeglasses:',len(list_eyeglasses_true))

df_attr_eyeglasses.head(10)

# Run test for Paper Figure 17

In [ ]:
run_selected_results= True
# run_selected_results= False

invert_heatmap_global = ['GradCAM','aIDG','aGradExpl','GradShapE']


In [ ]:
intrested_class_ls,intrested_class_lss

In [ ]:
methods_ls = [n for  n,_,_ in methods]
methods_ls

In [ ]:
# filter_methods = ['AA-500','Partition-500','BPT-500','LIME-500','GradCAM','aIDG']
filter_methods = methods_ls

filter_methods

## FUCNTIONS: PLotting

In [ ]:
from matplotlib import rc
rc('text',usetex=True)
rc('text.latex', preamble='\\usepackage{color}')


def fun_plot_heatmaps(heatmaps,methods_,path=None,title = None,plot_title=False,fontsize=14,plot_colorbar=False,save_fig=True,selected_ext='svg',dpi=200,transparent=True,destroy_fig=True):
    fig,axes = plt.subplots(1, len(methods_)+2, figsize=(2*(len(methods_)+2), 2))
    axes[0].imshow(image_to_explain)
    axes[0].set_xticks([]) ; axes[0].set_yticks([])
    axes[1].imshow(ground_truth, cmap='binary')
    axes[1].set_xticks([]) ; axes[1].set_yticks([])
    if plot_title:
        axes[0].set_title('Input', fontsize=fontsize)
        axes[1].set_title('GT', fontsize=fontsize)

    for ii, (n,c,_) in enumerate(methods_):
        ax = axes[ii+2]
        vmax = np.quantile(np.abs(heatmaps[n][0]), 0.99)
        ax.imshow(heatmaps[n][0], cmap=shap_bpt.shapley_values_colormap, vmin=-vmax, vmax=vmax)
        # ax.imshow(heatmaps[n][0], cmap=shap_bpt.colormap_default, vmin=-vmax, vmax=vmax)
        
        marked_h = mark_boundaries(np.tile((255,255,255,0), (heatmaps[n][0].shape[0],heatmaps[n][0].shape[1],1)), ground_truth, 
                             mode='thick', color=(0,0,0,1))
        ax.imshow(marked_h)
        ax.set_xticks([]) ; ax.set_yticks([])
        if plot_title:
            ax.set_title(n, fontsize=fontsize)
    plt.subplots_adjust(wspace=0.05,hspace=0.05)
    # plt.tight_layout(pad = 0.1)
    
    if save_fig:
        if title is None:
            suffix = f'{image_n}_{params.background_type}_heatmap.{selected_ext}'
        else:
            suffix = f'{image_n}_{params.background_type}_heatmap_{title}.{selected_ext}'
            # print('saving at',path)                                        
        plt.savefig(f'{path}//{suffix}',dpi=200,transparent=True,bbox_inches='tight', pad_inches=0.02)
    if destroy_fig:
        plt.close(fig)
    plt.show()
    del vmax,marked_h
    
props         = dict(boxstyle='round', facecolor='white', alpha=0.6, pad=0.2)
def fun_plot_IoU(heatmaps,IoU,methods_,path=None,title = None,plot_title=False,fontsize=14,save_fig=True,selected_ext='svg',dpi=200,transparent=True,destroy_fig=True):
    fig,axs = plt.subplots(1,  len(methods_)+2, figsize=(2*(len(methods_)+2), 2))
    axs[0].imshow(image_to_explain)
    axs[0].set_xticks([]) ; axs[0].set_yticks([])
    axs[1].imshow(ground_truth, cmap='binary')
    axs[1].set_xticks([]) ; axs[1].set_yticks([])
    if plot_title:
        axs[0].set_title('input', fontsize=fontsize)
        axs[1].set_title('GT', fontsize=fontsize)

    for ii, (n,_,_) in tqdm(enumerate(methods_), desc='IoU',leave=False):
        ax = axs[ii+2]
        img, max_IoU = vis_IoU(heatmaps[n][0], IoU[n][2], ground_truth), np.max(IoU[n][1])

        # print(np.min(img), np.max(img), img.shape, img.dtype)
        ax.imshow(img)
        ax.text(11,47, f'IoU:{max_IoU:.3}', fontsize=24, bbox=props,weight='bold')
        ax.set_xticks([]); ax.set_yticks([])
        if plot_title:
            ax.set_title(n, fontsize=fontsize)
    # plt.show()
    # plt.imshow(img)
    # return


    plt.subplots_adjust(wspace=0.05,hspace=0.05)
    # plt.tight_layout(pad = 0.1)
    if save_fig:
        if title is None:
            suffix = f'img_IoU.{selected_ext}'
        else:
            suffix = f'{image_n}_{params.background_type}_IoU_{title}.{selected_ext}'

        plt.savefig(f'{path}//{suffix}',dpi=dpi,transparent=transparent,bbox_inches='tight', pad_inches=0.02)
    if destroy_fig:
        plt.close(fig)
    plt.show()
    del img,max_IoU


#############################################################################


### fun_plot_performance

In [ ]:
# from matplotlib import rc
# rc('text',usetex=True)
# rc('text.latex', preamble='\\usepackage{color}')
# def fun_plot_performance(aucI, aucD, IoU,
#                          list_variants=['BPT-100', 'AA-100', 'LIME-100'],
#                          budget_set='100',
#                          path=None,
#                          file_name='',
#                          save_fig=True,
#                          fontsize=14,
#                          set_title=False,
#                          ttl=None,
#                          layout='row'  # 'row' or '2rows'
#                          ):
#     total_variant = 13
#     fig,axes = plt.subplots(1,total_variant, figsize=(25,2.5), sharex=True)#, sharey=True) #3,3  figsize=(8,2.2)
#     ##############################
#     aucI_aa      =  aucI[list_variants[0]]
#     aucD_aa      =  aucD[list_variants[0]] #['AA-100']
#     auc_IoU_aa   =  IoU [list_variants[0]] # ['AA-100']
#     ##############################
#     ttl = 'AA'
#     aucI_bpt     =  aucI[list_variants[1]]#['BPT-100']
#     aucD_bpt     =  aucD[list_variants[1]]   #['BPT-100']
#     auc_IoU_bpt  =  IoU[list_variants[1]]  ##['BPT-100']
#     ##############################
    
#     aucI_lime     =  aucI[list_variants[2]] ##['LIME-100']
#     aucD_lime     =  aucD[list_variants[2]]#['LIME-100']
#     auc_IoU_lime  =  IoU[list_variants[2]]#['LIME-100']
#     ###################################
#     def get_params_auc(auc_,typee):
#         if typee=='auc':
#             return auc_ ['xs'],auc_ ['ys'],auc_ ['ms'],auc_ ['auc']
#         elif typee=='auc_r':
#             return auc_ ['xs'],auc_ ['ysr'],auc_ ['ms'],auc_ ['auc_r']
#         ##############################
#         elif typee=='auc_adj':
#             return auc_ ['xs'],auc_ ['y_adj'],auc_ ['ms'],auc_ ['auc_adj']
#         elif typee=='auc_adj_r':
#             return auc_ ['xs'],auc_ ['y_adjr'],auc_ ['ms'],auc_ ['auc_adjr']
#         ##############################
#         elif typee=='auc_clip':
#             return auc_ ['xs'],auc_ ['y_clip'],auc_ ['ms'],auc_ ['auc_clip']
#         elif typee=='auc_clip_r':
#             return auc_ ['xs'],auc_ ['y_clipr'],auc_ ['ms'],auc_ ['auc_clipr']
#         ##############################
#         else:
#             print(f"{typee}-----> ERROR")
#     ###################################
#     def get_params_iou(auc_iou_,typee):
#         if typee=='auc_iou':
#             Xa,Ya,Ma,La = auc_iou_[0], auc_iou_[1], None, auc_iou_[4]
#             xma, yma = auc_iou_[3], np.max(auc_iou_[1])
#             return Xa,Ya,Ma,La, xma, yma

#             # return auc_ ['xs'],auc_ ['ys'],auc_ ['ms'],auc_ ['auc']
#             print(f"{typee}-----> ERROR")
        
#     ##############################
    
#     for i in range(total_variant):
#         ax = axes.flat[i]
#         ############  AUC
#         if i==0: # insertion/regression
#             # Xa,Ya,Ma,La      = aucI_aa['xs'],aucI_aa['ys'],aucI_aa['ms'],aucI_aa['auc_reg']
#             title='$\\mathit{AUC}^{+}$'
#             typee = 'auc'
#             Xa,Ya,Ma,La = get_params_auc(aucI_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucI_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucI_lime,typee)

#             # Xa,Ya,Ma,La      = aucI_aa ['xs'],aucI_aa ['ys'],aucI_aa ['ms'],aucI_aa ['auc']
#             # Xb,Yb,Mb,Lb      = aucI_bpt['xs'],aucI_bpt['ys'],aucI_bpt['ms'],aucI_bpt['auc']
#             # Xl,Yl,Ml,Ll      = aucI_lime['xs'],aucI_lime['ys'],aucI_lime['ms'],aucI_lime['auc']
#         elif i==1: # deletion/regression
#             title='$\\mathit{AUC}^{-}$'
#             typee = 'auc'
#             Xa,Ya,Ma,La = get_params_auc(aucD_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucD_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucD_lime,typee)

#             # Xa,Ya,Ma,La      = aucD_aa ['xs'],aucD_aa ['ys'],aucD_aa ['ms'],aucD_aa ['auc']
#             # Xb,Yb,Mb,Lb      = aucD_bpt['xs'],aucD_bpt['ys'],aucD_bpt['ms'],aucD_bpt['auc']


#         ############  AUC_Adj
#         elif i==2: # insertion/error
#             title='$\\mathit{AUC^{+}-Adj}$'
#             typee = 'auc_adj'
#             # Xa,Ya,Ma,La      = aucI_aa ['xs'],aucI_aa ['y_adj'],aucI_aa ['ms'],aucI_aa ['auc_adj']
#             # Xb,Yb,Mb,Lb      = aucI_bpt['xs'],aucI_bpt['y_adj'],aucI_bpt['ms'],aucI_bpt['auc_adj']
#             Xa,Ya,Ma,La = get_params_auc(aucI_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucI_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucI_lime,typee)

#         elif i==3: # deletion/error
#             typee = 'auc_adj'
#             title='$\\mathit{AUC^{-}-Adj}$'
#             Xa,Ya,Ma,La = get_params_auc(aucD_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucD_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucD_lime,typee)

#             # Xa,Ya,Ma,La = get_params_auc(aucD_aa, typee)
#             # Xb,Yb,Mb,Lb      = get_params_auc(aucD_bpt,typee)
#             # Xl,Yl,Ml,Ll      = get_params_auc(aucD_lime,typee)

#             # Xa,Ya,Ma,La      = aucD_aa ['xs'],aucD_aa ['y_adj'],aucD_aa ['ms'],aucD_aa ['auc_adj']
#             # Xb,Yb,Mb,Lb      = aucD_bpt['xs'],aucD_bpt['y_adj'],aucD_bpt['ms'],aucD_bpt['auc_adj']

#         ############  AUC_r
#         elif i==4: # insertion/error
#             title='$\\mathit{AUC^{+}-r}$'
#             typee = 'auc_r'
#             Xa,Ya,Ma,La = get_params_auc(aucI_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucI_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucI_lime,typee)
            
#             # Xa,Ya,Ma,La      = aucI_aa ['xs'],aucI_aa ['ysr'],aucI_aa ['ms'],aucI_aa ['auc_r']
#             # Xb,Yb,Mb,Lb      = aucI_bpt['xs'],aucI_bpt['ysr'],aucI_bpt['ms'],aucI_bpt['auc_r']            
        
#         elif i==5: # deletion/error
#             title='$\\mathit{AUC^{-}-r}$'
#             typee = 'auc_r'

#             Xa,Ya,Ma,La = get_params_auc(aucD_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucD_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucD_lime,typee)


#             # Xa,Ya,Ma,La      = aucD_aa ['xs'],aucD_aa ['ysr'],aucD_aa ['ms'],aucD_aa ['auc_r']
#             # Xb,Yb,Mb,Lb      = aucD_bpt['xs'],aucD_bpt['ysr'],aucD_bpt['ms'],aucD_bpt['auc_r']

    
#         ############  AUC_Adj_r
#         elif i==6: # insertion/error
#             title='$\\mathit{AUC^{+}-Adj-r}$'
#             typee = 'auc_adj_r'
#             Xa,Ya,Ma,La = get_params_auc(aucI_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucI_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucI_lime,typee)

#             # Xa,Ya,Ma,La      = aucI_aa ['xs'],aucI_aa ['y_adjr'],aucI_aa ['ms'],aucI_aa ['auc_adjr']
#             # Xb,Yb,Mb,Lb      = aucI_bpt['xs'],aucI_bpt['y_adjr'],aucI_bpt['ms'],aucI_bpt['auc_adjr']
#         elif i==7: # deletion/error
#             title='$\\mathit{AUC^{-}-Adj-r}$'
#             typee = 'auc_adj_r'
#             Xa,Ya,Ma,La = get_params_auc(aucD_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucD_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucD_lime,typee)
#             # Xa,Ya,Ma,La      = aucD_aa ['xs'],aucD_aa ['y_adjr'],aucD_aa ['ms'],aucD_aa ['auc_adjr']
#             # Xb,Yb,Mb,Lb      = aucD_bpt['xs'],aucD_bpt['y_adjr'],aucD_bpt['ms'],aucD_bpt['auc_adjr']
#         ########################################################################
#         elif i==8: # IoU
#             title='$\\mathit{AU}{\!-\!}\\mathit{IoU}$'
#             typee = 'auc_iou'
            
#             Xa,Ya,Ma,La,xma, yma = get_params_iou(auc_IoU_aa,typee)
#             Xb,Yb,Mb,Lb,xmb, ymb = get_params_iou(auc_IoU_bpt,typee)
#             # Xa,Ya,Ma,La = auc_IoU_aa[0], auc_IoU_aa[1], None, auc_IoU_aa[4]
#             # Xb,Yb,Mb,Lb = auc_IoU_bpt[0], auc_IoU_bpt[1], None, auc_IoU_bpt[4]
#             # xma, yma = auc_IoU_aa[3], np.max(auc_IoU_aa[1])
#             # xmb, ymb = auc_IoU_bpt[3], np.max(auc_IoU_bpt[1])
        

# ###############################################################

#         if i==9: # clipped insertion
#             title='Clipped $\\mathit{AUC}^{+}$'
#             typee = 'auc_clip'
#             Xa,Ya,Ma,La = get_params_auc(aucI_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucI_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucI_lime,typee)

#             # Xa,Ya,Ma,La = aucI_aa['xs'], aucI_aa['y_clip'], aucI_aa['ms'], aucI_aa['auc_clip']
#             # Xb,Yb,Mb,Lb = aucI_bpt['xs'], aucI_bpt['y_clip'], aucI_bpt['ms'], aucI_bpt['auc_clip']
#         elif i==10: # clipped deletion
#             title='Clipped $\\mathit{AUC}^{-}$'
#             typee = 'auc_clip'
#             Xa,Ya,Ma,La = get_params_auc(aucD_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucD_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucD_lime,typee)

#             # Xa,Ya,Ma,La = aucD_aa['xs'], aucD_aa['y_clip'], aucD_aa['ms'], aucD_aa['auc_clip']
#             # Xb,Yb,Mb,Lb = aucD_bpt['xs'], aucD_bpt['y_clip'], aucD_bpt['ms'], aucD_bpt['auc_clip']
#         elif i==11: # clipped rescaled insertion
#             title='Clipped rescaled $\\mathit{AUC}^{+}$'
#             typee = 'auc_clip_r'
#             Xa,Ya,Ma,La = get_params_auc(aucI_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucI_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucI_lime,typee)

#             # Xa,Ya,Ma,La = aucI_aa['xs'], aucI_aa['y_clipr'], aucI_aa['ms'], aucI_aa['auc_clipr']
#             # Xb,Yb,Mb,Lb = aucI_bpt['xs'], aucI_bpt['y_clipr'], aucI_bpt['ms'], aucI_bpt['auc_clipr']
#         elif i==12: # clipped rescaled deletion
#             title='Clipped rescaled $\\mathit{AUC}^{-}$'
#             typee = 'auc_clip_r'
#             Xa,Ya,Ma,La = get_params_auc(aucD_aa, typee)
#             Xb,Yb,Mb,Lb      = get_params_auc(aucD_bpt,typee)
#             Xl,Yl,Ml,Ll      = get_params_auc(aucD_lime,typee)


#             # Xa,Ya,Ma,La = aucD_aa['xs'], aucD_aa['y_clipr'], aucD_aa['ms'], aucD_aa['auc_clipr']
#             # Xb,Yb,Mb,Lb = aucD_bpt['xs'], aucD_bpt['y_clipr'], aucD_bpt['ms'], aucD_bpt['auc_clipr']
    




#         # print(i,'  len(Xb): ',len(Xb),'  len(Yb): ', len(Yb), len(Xb)==len(Yb), '  len(Xa): ',len(Xa),'  len(Ya): ', len(Ya), len(Xa)==len(Ya))
#     #     ymax = max(np.max(Ya), np.max(Yb))
        
#         Sa, Sb = ('\\textbf', '') if La<Lb else ('', '\\textbf')
#         if i in [0,2,4,6, 8]:   Sa,Sb=Sb,Sa

#         # print('La',La)
#         ax.plot(Xa, Ya, c='#2465a6', label=f'{Sa}{{AA}} {La:.4}')
#         ax.fill_between(Xa, Ya, color='#2465a6', alpha=0.15, hatch='///')
#         # ax.scatter(Xa, Ya, c='black', s=5)
#         ax.plot(Xb, Yb, c='#9d2f4d', label=f'{Sb}{{BPT}} {Lb:.4}', alpha=0.80)
#         ax.fill_between(Xb, Yb, color='#9d2f4d', alpha=0.15, hatch='\\\\\\')
        
#         if i==8:
#             ax.scatter(xma, yma, s=40, color='blue')
#             ax.scatter(xmb, ymb, s=40, color='red')

#         if i < 4:
#             ax.axhline(abs(f_S-f_0), ls='--', c='grey', zorder=0)
            
#         ax.axhline(0, c='lightgrey', zorder=0)
#         ax.legend(borderpad=0.2, labelspacing=0.1, loc='upper right' if i>=1 else 'lower right')#, bbox_to_anchor=(1,0))
#         ax.set_title(title, fontsize=fontsize)
    
#     axes[0].set_xlabel('\% pixels inserted', fontsize=14)
#     axes[1].set_xlabel('\% pixels deleted', fontsize=14)
#     axes[2].set_xlabel('\% pixels inserted', fontsize=14)
#     axes[3].set_xlabel('\% pixels deleted', fontsize=14)
#     if set_title:
#         plt.suptitle(f'{ttl}')
#     plt.tight_layout()
#     if save_fig:
#         if path is not None:
#             plt.savefig(f'{path}/{file_name}_image_five_metrics__2.png', dpi=150, bbox_inches='tight', pad_inches=0.02)
#         else:
#             print('path is None')
#     # plt.savefig(f'{path}/five_metrics_{background_type}_2.svg', dpi=150, bbox_inches='tight', pad_inches=0.02)
    
#     plt.show()

In [ ]:
from matplotlib import rc
rc('text',usetex=True)
rc('text.latex', preamble='\\usepackage{color}')

def fun_plot_performance(aucI, aucD, IoU,
                         list_variants=['BPT-100', 'AA-100', 'LIME-100'],
                         budget_set='100',
                         path=None,
                         file_name='',
                         save_fig=True,
                         fontsize=14,
                         set_title=False,
                         plot_lime=False,
                         ttl=None,
                         layout='row'  # 'row' or '2rows'
                         ):
    import numpy as np
    import matplotlib.pyplot as plt

    # Order of plots (grouped by column for 2-row layout)
    plot_defs = [
        {'title': '$\\mathit{AUC}^{+}$',         'typee': 'auc',        'from': 'aucI'}, # 0
        {'title': '$\\mathit{AUC}^{-}$',         'typee': 'auc',        'from': 'aucD'}, # 1
        {'title': '$\\mathit{AUC}^{+}-clip$',     'typee': 'auc_clip',   'from': 'aucI'}, # 2
        {'title': '$\\mathit{AUC}^{-}-clip$',     'typee': 'auc_clip',   'from': 'aucD'}, # 3
        {'title': '$\\mathit{AUC^{+}-Adj}$',     'typee': 'auc_adj',    'from': 'aucI'}, # 4
        {'title': '$\\mathit{AUC^{-}-Adj}$',     'typee': 'auc_adj',    'from': 'aucD'}, # 5
        
        {'title': '$\\mathit{AU}{\!-\!}\\mathit{IoU}$', 'typee': 'auc_iou', 'from': 'iou'}, # 6 
        #####################################################################

        {'title': '$\\mathit{AUC}^{+}-r$',       'typee': 'auc_r',      'from': 'aucI'}, # 7
        {'title': '$\\mathit{AUC}^{-}-r$',       'typee': 'auc_r',      'from': 'aucD'}, # 8
        {'title': '$\\mathit{AUC}^{+}-clip-r$', 'typee': 'auc_clip_r', 'from': 'aucI'}, # 9
        {'title': '$\\mathit{AUC}^{-}-clip-r$', 'typee': 'auc_clip_r', 'from': 'aucD'}, # 10
        {'title': '$\\mathit{AUC^{+}-Adj-r}$',   'typee': 'auc_adj_r',  'from': 'aucI'}, # 11
        {'title': '$\\mathit{AUC^{-}-Adj-r}$',   'typee': 'auc_adj_r',  'from': 'aucD'}, # 12 
        
    ]
    
    # Reorder plot_defs for 2-row layout: interleaved columns
    if layout == '2rows':
        top = plot_defs[0::2]
        bottom = plot_defs[1::2]
        plot_defs = [None]*(len(top)+len(bottom))
        plot_defs[::2] = top
        plot_defs[1::2] = bottom

    total_variant = len(plot_defs)

    # Layout config
    if layout == 'row':
        nrows, ncols = 1, total_variant
        fig_width, fig_height = 25, 2.5
    elif layout == '2rows':
        nrows, ncols = 2, int(np.ceil(total_variant / 2))
        fig_width, fig_height = 16, 5
    else:
        raise ValueError("layout must be either 'row' or '2rows'")

    fig, axes = plt.subplots(nrows, ncols, figsize=(fig_width, fig_height))
    axes = axes.flatten()

    # Alias
    aucI_aa = aucI[list_variants[0]]
    aucD_aa = aucD[list_variants[0]]
    auc_IoU_aa = IoU[list_variants[0]]

    aucI_bpt = aucI[list_variants[1]]
    aucD_bpt = aucD[list_variants[1]]
    auc_IoU_bpt = IoU[list_variants[1]]

    aucI_lime = aucI[list_variants[2]]
    aucD_lime = aucD[list_variants[2]]
    auc_IoU_lime = IoU[list_variants[2]]

    def get_params_auc(auc_, typee):
        return {
            'auc':        (auc_['xs'], auc_['ys'],     auc_['ms'], auc_['auc']),
            'auc_r':      (auc_['xs'], auc_['ysr'],    auc_['ms'], auc_['auc_r']),
            'auc_adj':    (auc_['xs'], auc_['y_adj'],  auc_['ms'], auc_['auc_adj']),
            'auc_adj_r':  (auc_['xs'], auc_['y_adjr'], auc_['ms'], auc_['auc_adjr']),
            'auc_clip':   (auc_['xs'], auc_['y_clip'], auc_['ms'], auc_['auc_clip']),
            'auc_clip_r': (auc_['xs'], auc_['y_clipr'],auc_['ms'], auc_['auc_clipr']),
        }.get(typee, (None, None, None, None))

    def get_params_iou(iou_, typee):
        if typee == 'auc_iou':

            # xmb, ymb = iou_[3], np.max(iou_[1])


            Xa, Ya, Ma, La = iou_[0], iou_[1], None, iou_[4]
            xma, yma = iou_[3], np.max(iou_[1])
            return Xa, Ya, Ma, La, xma, yma
        return None, None, None, None, None, None

    for i, config in enumerate(plot_defs):
        ax = axes[i]
        title = config['title']
        typee = config['typee']
        source = config['from']

        # Select correct dict
        if source == 'aucI':
            Xa, Ya, Ma, La = get_params_auc(aucI_aa, typee)
            Xb, Yb, Mb, Lb = get_params_auc(aucI_bpt, typee)
            Xl, Yl, Ml, Ll = get_params_auc(aucI_lime, typee)
        elif source == 'aucD':
            Xa, Ya, Ma, La = get_params_auc(aucD_aa, typee)
            Xb, Yb, Mb, Lb = get_params_auc(aucD_bpt, typee)
            Xl, Yl, Ml, Ll = get_params_auc(aucD_lime, typee)
        elif source == 'iou':
            Xa, Ya, Ma, La, xma, yma = get_params_iou(auc_IoU_aa,  typee)
            Xb, Yb, Mb, Lb, xmb, ymb = get_params_iou(auc_IoU_bpt, typee)
            Xl, Yl, Ml, Ll, xml, yml = get_params_iou(auc_IoU_lime,   typee)

        # Boldness logic
        Sa, Sb = ('\\textbf', '') if La < Lb else ('', '\\textbf')
        if i in [0,2,4,6,8]: Sa, Sb = Sb, Sa

        # Plot
        ax.plot(Xa, Ya, c='#2465a6', label=f'{Sa}{{AA}} {La:.4f}')
        ax.fill_between(Xa, Ya, color='#2465a6', alpha=0.15, hatch='///')
        ##########
        ax.plot(Xb, Yb, c='#9d2f4d', label=f'{Sb}{{BPT}} {Lb:.4f}', alpha=0.80)
        ax.fill_between(Xb, Yb, color='#9d2f4d', alpha=0.15, hatch='\\\\\\')
        ##########
        if plot_lime:
            ax.plot(Xl, Yl, c="#01B0A7", label=f'{Sb}{{LM}} {Ll:.4f}', alpha=0.80)
            ax.fill_between(Xl, Yl, color='#01B0A7', alpha=0.15, hatch='\\\\\\')


        if typee == 'auc_iou':
            ax.scatter(xma, yma, s=40, color='blue')
            ax.scatter(xmb, ymb, s=40, color='red')

        if i < 4:
            ax.axhline(0.0, ls='--', c='grey', zorder=0)  # placeholder for f_S - f_0

        ax.axhline(0, c='lightgrey', zorder=0)
        ax.legend(borderpad=0.2, labelspacing=0.1, loc='upper right' if i>=1 else 'lower right')
        ax.set_title(title, fontsize=fontsize)
    axes[-1].imshow(image_to_explain); axes[-1].set_xticks([]); axes[-1].set_yticks([])
    axes[-1].set_title(ttl.split(",")[0])
    if set_title:
        plt.suptitle(f'{ttl}', fontsize=16)
    plt.tight_layout()

    if save_fig and path is not None:
        plt.savefig(f'{path}/{file_name}_paired_auc_metrics.png', dpi=150, bbox_inches='tight', pad_inches=0.02)
    plt.show()


In [ ]:
from matplotlib import rc
rc('text',usetex=True)
rc('text.latex', preamble='\\usepackage{color}')

In [ ]:
filter_hairs = ['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']

## RUN:

In [ ]:
# intrested_class_ls, intrested_class_lss
# img_id



def get_real_class(image_id,df_attr,filter_hairs, verbose=False):

    # Optional: check which of the desired hair columns actually exist in your DataFrame
    existing_hair_cols = [col for col in filter_hairs if col in df_attr.columns]

    # Now, get the values for a specific image_id
    row = df_attr[df_attr.image_id == image_id][existing_hair_cols]

    # If you want it as a dictionary (column: value)
    hair_values = row.iloc[0].to_dict()
    if verbose:
        print('hair_values : ', hair_values)

    dominant_hair = [k for k, v in hair_values.items() if v == 1][0]
    if verbose:
        print('dominant_hair: ', dominant_hair)
    return dominant_hair

# def get_real_class_eye(image_id,df_attr,filter_hairs, verbose=False):

#     # Optional: check which of the desired hair columns actually exist in your DataFrame
#     existing_hair_cols = [col for col in filter_hairs if col in df_attr.columns]

#     # Now, get the values for a specific image_id
#     row = df_attr[df_attr.image_id == image_id][existing_hair_cols]

#     # If you want it as a dictionary (column: value)
#     hair_values = row.iloc[0].to_dict()
#     if verbose:
#         print('hair_values : ', hair_values)

#     dominant_hair = [k for k, v in hair_values.items() if v == 1][0]
#     if verbose:
#         print('dominant_hair: ', dominant_hair)
#     return dominant_hair

In [ ]:
run_selected_results= True
# run_selected_results= False

In [ ]:
if run_selected_results:
    ##################################################    
    # files_selected = [18,32,36,43,47] # 18 , 27, 32
    # files_selected = [32] # 18 , 27, 32
    files_selected = [7] # high 65  
    ##################################################
    start_time = datetime.now()
    print(f'{"results saved at" :<20}  {results_selected}')
    print(f'{"code started"     :<20}  {start_time      }')
    print(f'{"background_type"  :<20}  {background_type     :<10}')
    print(f'{"Total Images"     :<20}  {len(files_selected) :<10}')
    print(f'{"img_id"           :<10}  {"classes_name":<15}  {"f_S":<10}  {"f_0":<10}  {"f_S-f_0":<10}')
    
    for img_id in tqdm(files_selected, leave=False, desc='Images : '):
        # img_id = int(img_id.split('.')[0].strip("0"))
        img_filename = f'{ds_imgs_path}/{img_id}.jpg'
        image_id = f'{img_id}.jpg'
        results_selected_paperfig = os.path.join(results_selected,str(img_id))
        os.makedirs(results_selected_paperfig,          exist_ok=True)
        
        for intrested_class,intrested_class_lbl in zip(intrested_class_ls,intrested_class_lss):
            ####################
            if intrested_class_lbl=='Eyeglasses':
                continue
            ####################
            load_image_to_explain(intrested_class,img_id=img_id,bg_type=background_type)
            # load_ground_truth(img_id)
            load_ground_truth(intrested_class_lss=[intrested_class_lbl],img_id=img_id)
            ground_truth_final = union_ground_truths[classes_names[intrested_class]]==255
            ################################################
            true_intrested_class = df_attr[df_attr.image_id==image_id][intrested_class_lbl].values[0]==1
            
            dominant_hair = get_real_class(image_id,df_attr,filter_hairs)
            
            ################################################
            invert_heatmap = (predicted_fS[intrested_class] < 0)
            #############################################
            sorted_classes = np.array([intrested_class])
            print(f'{img_id} \t {classes_names[intrested_class]:<10}? == {true_intrested_class}, f_S [{f_S:0.6}] | f_0 [{f_0:0.6}]' )
            heatmaps = {}
            aucD, aucI, IoU = {}, {}, {}
            for n,_,funct in tqdm(methods, leave=False):
                if n not in filter_methods:
                    continue
                
                heatmaps[n] = funct()
                aucD[n] = saliency_to_auc(f_masked,heatmaps[n][0],f_S, f_0, predicted_cls=0, method='del', batch_size=batch_size)
                aucI[n] = saliency_to_auc(f_masked,heatmaps[n][0],f_S, f_0, predicted_cls=0, method='ins', batch_size=batch_size)
                IoU[n]  = calc_IoU_curve(ground_truth_final.flatten(), heatmaps[n][0].flatten())
            plot_heatmaps(image_to_explain,ground_truth_final,heatmaps,img_id,save_dir=results_selected_paperfig,plot_title=True,
                          save_figs=True, annotate_text=False, ext='svg', dpi=100)
            print(f'f_S: [{f_S}] f_0: [{f_0}]')
            f_S_ones = f_masked(np.ones((1,224,224)).astype(bool))[0][0]
            f_0_zeros = f_masked(np.zeros((1,224,224)).astype(bool))[0][0]
            print('f_S_ones: ',f_S_ones)
            print('f_0_zeros: ',f_0_zeros)
            plt_ttl = f'img_id: {img_id}, explained_class: {intrested_class_lbl} '\
                        f'True Class: {dominant_hair} \n'\
                        f'f_S:\ {f_S:0.6}, f_0: {f_0:0.6}, delta: {abs(f_S-f_0):0.6}\n'\
                        f'f_S_ones: {f_S_ones:0.6}, f_0_zeros: {f_0_zeros:0.6}'
            # fun_plot_performance(aucI,aucD,IoU,path=results_selected_paperfig, file_name=img_id, layout='2rows')
            fun_plot_performance(aucI,aucD,IoU,path=results_selected_paperfig, file_name=img_id,layout='2rows',
                     set_title=True,plot_lime=False,
                     ttl=plt_ttl)
            ###########################
            plot_filtered_IoU(heatmaps,ground_truth_final,invert_heatmap,invert_heatmap_global,IoU,img_id,filter_methods=filter_methods,
                              save_dir=results_selected_paperfig,
                              save_figs=True,ext='svg',destroy_fig=False)

## PLOT HEATMAPS

In [ ]:
plot_heatmaps(image_to_explain,ground_truth_final,heatmaps,img_id,save_dir=results_selected_paperfig,plot_title=False,
                          save_figs=True, annotate_text=False, ext='svg', dpi=100)

## Visualize IoU and GT

In [ ]:
plot_title = True
save_fig   = True
# get_exp_header()
# fun_plot_IoU(heatmaps,IoU,methods,path='',destroy_fig=False, selected_ext='png',plot_title=plot_title, save_fig=save_fig,fontsize=12)

## PLOT AUCs

In [ ]:
# get_exp_header()
if run_selected_results:
    dominant_hair = get_real_class(image_id,df_attr,filter_hairs, verbose=True)
    plot_heatmaps(image_to_explain,ground_truth_final,heatmaps,img_id,save_dir=results_selected_paperfig,plot_title=True,
                            save_figs=True, annotate_text=False, ext='png', dpi=100)
    # print(f'f_S:{f_S}\nf_0:{f_0}' )
    # print('ones',f_masked(np.ones((1,224,224)).astype(bool)))
    # print('zeeros',f_masked(np.zeros((1,224,224)).astype(bool)))
    plt_ttl = f'img_id: {img_id}, explained_class: {intrested_class_lbl} '\
                f'True Class: {dominant_hair} \n'\
                f'f_S:\ {f_S:0.6}, f_0: {f_0:0.6}, delta: {abs(f_S-f_0):0.6}\n'\
                f'f_S_ones: {f_S_ones:0.6}, f_0_zeros: {f_0_zeros:0.6}'
    # fun_plot_performance(aucI,aucD,IoU,path=results_selected_paperfig, file_name=img_id, layout='2rows')
    fun_plot_performance(aucI,aucD,IoU,path=results_selected_paperfig, file_name=img_id,layout='2rows',
                set_title=True,plot_lime=True,
                ttl=plt_ttl)

In [ ]:
run_selected_results = True

In [ ]:
if run_selected_results:
    destroy_fig         = True
    
    ##################################################
    # files_selected = [821 ,1548,1330,5426,36,43,47,52,55,61,64]    # for paper
    # files_selected = [32,95,132]    # positives and negatives
    ##################################################
    # files_selected = [7]
    # FOR FIGURE NO 14 --- 
    files_selected = [32,95,132, 821, 1330, 64]    # positives and negatives
    ##################################################
    
    # files_selected = [95] # 18 , 27, 32
    #files_selected = [52,55,61,64] # 18 , 27, 32
    # files_selected = [52,821] # 18 , 27, 32
    ##################################################

    start_time = datetime.now()
    print(f'{"results saved at" :<20}  {results_selected}')
    print(f'{"code started"     :<20}  {start_time      }')
    print(f'{"background_type"  :<20}  {background_type     :<10}')
    print(f'{"Total Images"     :<20}  {len(files_selected) :<10}')
    print(f'{"img_id"           :<10}  {"classes_name":<15}  {"f_S":<10}  {"f_0":<10}  {"f_S-f_0":<10}')
    
    for img_id in tqdm(files_selected, leave=False, desc='Images : '):
        # img_id = int(img_id.split('.')[0].strip("0"))
        img_filename = f'{ds_imgs_path}/{img_id}.jpg'
        image_id = f'{img_id}.jpg'
        results_selected_paperfig = os.path.join(results_selected,str(img_id))
        os.makedirs(results_selected_paperfig,          exist_ok=True)
        
        sorted_classes = intrested_class_ls
        # load_image_to_explain(img_id=img_id,bg_type=background_type)           
        
        for  intrested_class,intrested_class_lbl in zip(intrested_class_ls,intrested_class_lss):
            # if abs(predicted_fS[intrested_class])<1: 
            #     continue;
            load_image_to_explain(intrested_class,img_id=img_id,bg_type=background_type)
            # load_ground_truth(img_id)
            load_ground_truth(intrested_class_lss=[intrested_class_lbl],img_id=img_id)
            
            # load_ground_truth(img_id)
            ground_truth_final = union_ground_truths[classes_names[intrested_class]]==255
            ################################################
            true_intrested_class = df_attr[df_attr.image_id==image_id][intrested_class_lbl].values[0]==1
            if intrested_class_lbl=='Brown_Hair':
                dominant_hair = get_real_class(image_id,df_attr,filter_hairs)
            else:
                dominant_hair = 'Eyeglasses?'
            # dominant_hair = get_real_class(image_id,df_attr,filter_hairs)
            #####################################################
            
            invert_heatmap = (predicted_fS[intrested_class] < 0)

            ################################################
            #sorted_classes = np.array([intrested_class])
            plot_img_by_parts(save_dir=results_selected_paperfig, dpi=100)
            print(f'{img_id} | {classes_names[intrested_class]:<10}? == {not invert_heatmap} | {predicted_fS[intrested_class]:0.5}')
            
            print(f'f_S: [{f_S}] f_0: [{f_0}]')
            print('-'*100)

In [ ]:
if run_selected_results:
    destroy_fig         = True
    save_fig_parts      = False
    
    ##################################################
    # files_selected = [821 ,1548,1330,5426,36,43,47,52,55,61,64]    # for paper
    # files_selected = [32,95,132]    # positives and negatives
    ##################################################
    # files_selected = [7]
    # FOR FIGURE NO 14 --- 
    files_selected = [32,95,132, 821, 1330, 64]    # positives and negatives
    ##################################################
    
    # files_selected = [95] # 18 , 27, 32
    #files_selected = [52,55,61,64] # 18 , 27, 32
    # files_selected = [52,821] # 18 , 27, 32
    ##################################################

    start_time = datetime.now()
    print(f'{"results saved at" :<20}  {results_selected}')
    print(f'{"code started"     :<20}  {start_time      }')
    print(f'{"background_type"  :<20}  {background_type     :<10}')
    print(f'{"Total Images"     :<20}  {len(files_selected) :<10}')
    print(f'{"img_id"           :<10}  {"classes_name":<15}  {"f_S":<10}  {"f_0":<10}  {"f_S-f_0":<10}')
    
    for img_id in tqdm(files_selected, leave=False, desc='Images : '):
        # img_id = int(img_id.split('.')[0].strip("0"))
        img_filename = f'{ds_imgs_path}/{img_id}.jpg'
        image_id = f'{img_id}.jpg'
        results_selected_paperfig = os.path.join(results_selected,str(img_id))
        os.makedirs(results_selected_paperfig,          exist_ok=True)
        
        sorted_classes = intrested_class_ls
        # load_image_to_explain(img_id=img_id,bg_type=background_type)           
        
        for  intrested_class,intrested_class_lbl in zip(intrested_class_ls,intrested_class_lss):
            # if abs(predicted_fS[intrested_class])<1: 
            #     continue;
            load_image_to_explain(intrested_class,img_id=img_id,bg_type=background_type)
            # load_ground_truth(img_id)
            load_ground_truth(intrested_class_lss=[intrested_class_lbl],img_id=img_id)
            
            # load_ground_truth(img_id)
            ground_truth_final = union_ground_truths[classes_names[intrested_class]]==255
            ################################################
            true_intrested_class = df_attr[df_attr.image_id==image_id][intrested_class_lbl].values[0]==1
            if intrested_class_lbl=='Brown_Hair':
                dominant_hair = get_real_class(image_id,df_attr,filter_hairs)
            else:
                dominant_hair = 'Eyeglasses?'
            # dominant_hair = get_real_class(image_id,df_attr,filter_hairs)
            #####################################################
            
            invert_heatmap = (predicted_fS[intrested_class] < 0)

            ################################################
            #sorted_classes = np.array([intrested_class])
            plot_img_by_parts(save_dir=results_selected_paperfig, dpi=100)
            print(f'{img_id} | {classes_names[intrested_class]:<10}? == {not invert_heatmap} | {predicted_fS[intrested_class]:0.5}')
            
            heatmaps = {}
            aucD, aucI, IoU = {}, {}, {}
            for n,_,funct in tqdm(methods, leave=False):
                if n not in filter_methods:
                    continue;
                heatmaps[n] = funct()
                if save_fig_parts:
                    plot_heatmaps_by_parts(save_dir=results_selected_paperfig, dpi=100)
            #######################################################################
            # plot_heatmaps(image_to_explain,ground_truth_final,heatmaps,image_to_explain,ground_truth_final,heatmaps,img_id,save_dir=results_selected_paperfig,plot_title=True, save_figs=True, annotate_text=True, )
            # plot_heatmaps_svg(img_id,save_dir=results_selected_paperfig,plot_title=False, save_figs=True,
            #               annotate_text=False, ext='svg',destroy_fig=True)
            plot_filtered_heatmaps(heatmaps,ground_truth_final,invert_heatmap,invert_heatmap_global,img_id,filter_methods=filter_methods,save_dir=results_selected_paperfig,plot_title=False,bg_type=background_type,
                                   save_figs=True, annotate_text=False, ext='svg', dpi=100)
            plot_filtered_heatmaps(heatmaps,ground_truth_final,invert_heatmap,invert_heatmap_global,img_id,filter_methods=filter_methods,save_dir=results_selected_paperfig,
                                   plot_title=True,bg_type=background_type,destroy_fig=True,
                                   save_figs=True, annotate_text=False, ext='png', dpi=100)
            print(f'f_S: [{f_S}] f_0: [{f_0}]')
            f_S_ones = f_masked(np.ones((1,224,224)).astype(bool))[0][0]
            f_0_zeros = f_masked(np.zeros((1,224,224)).astype(bool))[0][0]
            print('f_S_ones: ',f_S_ones)
            print('f_0_zeros: ',f_0_zeros)
            ################################  IOU  ################################
            for n,_,funct in tqdm(methods, leave=False):
                if n not in filter_methods:
                    continue
                #######################################################################
                # FOR NEGATIVE SHAPELY VALUES
                heatmaps[n]=-heatmaps[n] if np.sum(heatmaps[n])<0 else heatmaps[n]
                hmap = heatmaps[n]
                if invert_heatmap:
                    hmap = np.abs(heatmaps[n])
                if n in invert_heatmap_global:
                    # print('abs values for ',n)
                    hmap = np.abs(heatmaps[n])
                
                aucD[n] = saliency_to_auc(f_masked,heatmaps[n][0],f_S, f_0, predicted_cls=0, method='del', batch_size=batch_size)
                aucI[n] = saliency_to_auc(f_masked,heatmaps[n][0],f_S, f_0, predicted_cls=0, method='ins', batch_size=batch_size)
                IoU[n]  = calc_IoU_curve(ground_truth_final.flatten(), hmap[0].flatten())
            
            # fun_plot_performance(aucI,aucD,IoU,path=results_selected_paperfig)
            plt_ttl = f'img_id: {img_id}, explained_class: {intrested_class_lbl}, '\
                        f'True Class: {dominant_hair} \n'\
                        f'f_S:\ {f_S:0.6}, f_0: {f_0:0.6}, delta: {abs(f_S-f_0):0.6}\n'\
                        f'f_S_ones: {f_S_ones:0.6}, f_0_zeros: {f_0_zeros:0.6}'
            # fun_plot_performance(aucI,aucD,IoU,path=results_selected_paperfig, file_name=img_id, layout='2rows')
            fun_plot_performance(aucI,aucD,IoU,path=results_selected_paperfig, file_name=img_id,layout='2rows',
                                 set_title=True,
                                 ttl=plt_ttl)
            if save_fig_parts:
                plot_iou_by_parts(save_dir=results_selected_paperfig, dpi=100)
            #############################################
            # plot_IoU(img_id,save_dir=results_selected_paperfig, invert_heatmap=invert_heatmap,save_figs=True, ext='png')
            # plot_IoU_svg(img_id,save_dir=results_selected_paperfig,invert_heatmap=invert_heatmap, save_figs=True,ext='svg',destroy_fig=True)
            plot_filtered_IoU(heatmaps,ground_truth_final,invert_heatmap,invert_heatmap_global,IoU,img_id,filter_methods=filter_methods,
                              save_dir=results_selected_paperfig,
                              save_figs=True,ext='svg',destroy_fig=False)
            plot_filtered_IoU(heatmaps,ground_truth_final,invert_heatmap,invert_heatmap_global,IoU,img_id,filter_methods=filter_methods,
                            save_dir=results_selected_paperfig,
                            plot_title = True,
                            save_figs=True,ext='png',destroy_fig=True)
            # break
        # break
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

## Test Background Senstivity for Single Image


In [ ]:
TEST_BG_SEN = False

In [ ]:
if TEST_BG_SEN:
    files_selected      = [18,32,36,43,47] # 18 , 27, 32
    background_type_ls  = ['full','black' , 'white' , 'gray' , 'noise'  ,  'blurred']

    print(f'{"img_id"          :<10}  {"classes_name":<15}  {"f_S":<10}  {"f_0":<10}  {"f_S-f_0":<10}')
    f_S_ls = []

    for img_id in tqdm(files_selected, leave=False, desc='Images : '):
        for background_typee_id,background_typee in enumerate(tqdm(background_type_ls, desc='Bg Types : ')):
            img_filename = f'{ds_imgs_path}/{img_id}.jpg'
            
            for intrested_class,intrested_class_lbl in zip(intrested_class_ls,intrested_class_lss):
                image_id = f'{img_id}.jpg'
                #print(intrested_class_lbl)
                true_intrested_class = df_attr[df_attr.image_id==image_id][intrested_class_lbl].values[0]==1
                #print('true_intrested_class\t',true_intrested_class)
                sorted_classes = np.array([intrested_class])
                
                # load_image_to_explain(img_id=img_id,bg_type=background_typee)
                load_image_to_explain(intrested_class,img_id=img_id,bg_type=background_type)
                load_ground_truth(img_id)

                true_intrested_class = df_attr[df_attr.image_id==image_id][intrested_class_lbl].values[0]==1
                
                dominant_hair = get_real_class(image_id,df_attr,filter_hairs)

                
                print(f'{img_id:<10} {background_typee:<10} {classes_names[intrested_class]:<10}\t{f_S:<10.5} \t {f_0:<10.5} {f_S-f_0:<10.5}')
                
                f_S_ls.append({'img_id':img_id,
                            'background_type':background_typee,
                            'intrested_class':intrested_class,
                            'intrested_class_lbl':intrested_class_lbl,
                            'true_intrested_class':true_intrested_class,
                            'f_S':f_S,
                            'f_0':f_0,
                            'f_S-f_0':f_S-f_0,
                            })
                ground_truth_final = union_ground_truths[classes_names[intrested_class]]==255
                sorted_classes = np.array([intrested_class])
            if background_typee_id==0:
                plt.subplot(121),plt.imshow(image_to_explain)
                plt.subplot(122),plt.imshow(ground_truth_final, cmap='binary')
                plt.show()
        print('='*120)
    dfff = pd.DataFrame(f_S_ls)
    dfff.head()

# Compute Full Results for All Selected Images with Selected BG
**---NOTE---** Switch `run_full_test` to True to reproduce all the results for all the images

In [ ]:
len(list_brownhairs)

In [ ]:
#run_full_test = False
run_full_test = True

# list_eyeglasses_true - list_brownhairs
subfile_count = 1000

files_sub,filter_type  = list_brownhairs[:subfile_count],'brownhairs'
#files_sub,filter_type  = list_eyeglasses_true[:200],'eyeglasses_true'

background_type        = 'gray'     # black , white , gray , noise , blurred , full

In [ ]:
print(f'tota images with {filter_type.upper()} are {len(list_brownhairs)}')
print(f'running for {len(files_sub)}')

In [ ]:
def get_bg_values(f_masked,ground_truth,predicted_cls,class_names):
    global f_G, f_B
    # evaluate the ground truth mask with the background replacement strategy for masking function
    predicted_fG = f_masked(np.expand_dims(ground_truth, axis=0))[0]
    f_G = float(predicted_fG[predicted_cls])
    print(class_names[predicted_cls], f_G, predicted_cls, f_G)
    # print('softmax prob:', np_softmax(predicted_fG)[predicted_cls])

    # evaluate the backgrounf (negative of the ground truth mask)
    background_mask = np.logical_not(ground_truth)
    predicted_fB = f_masked(np.expand_dims(background_mask, axis=0))[0]
    f_B = float(predicted_fB[predicted_cls])
    print(class_names[predicted_cls], f_B, predicted_cls, f_B)
    # print('softmax prob:', np_softmax(predicted_fB)[predicted_cls])

    print()
    print('nu(S):  ', round(f_S, 4))
    print('nu(G):  ', round(f_G, 4))
    print('nu(S/G):', round(f_B, 4))
    print('nu(0):  ', round(f_0, 4))

In [ ]:
import pickle 

vector_path                    = os.path.join(results_path,'vectors')

In [ ]:
if run_full_test:
    verbose_print    = False
    save_vectors_heatmaps  = False
    disallowed_vectors = []
    
    total_time_st = time.time()
    start_time = datetime.now()
    
    print(f'code started:\t{start_time}')
    iterations = len(files_sub)
    csv_filename = f'{path_csv}/csv_exp_{exp_type}_IoU_face_{iterations}_{len(methods)}_{background_type}_{filter_type}.csv'
    print('csv_filename : ',csv_filename)

    data_to_csv=[]
    print(f'{"results saved at" :<20}  {results_path        }')
    print(f'{"code started"     :<20}  {start_time              }')
    print(f'{"background_type"  :<20}  {background_type     :<10}')
    print(f'{"Total Images"     :<20}  {iterations          :<10}')
    print(f'{"img_id"           :<10}  {"classes_name":<15}  {"f_S":<10}  {"f_0":<10}  {"f_S-f_0":<10}')
    #######################
    sorted_classes = intrested_class_ls 
    #######################
    first_exp_print = True
    first_exp_plot = True
    for image_id in tqdm(files_sub, leave=False, desc='Images : '):
        #if att_csv_celeba:
            #img_id = int(img_id.split('.')[0].strip("0"))   # for att_csv_celeba 
        
        img_id = int(image_id.split('.')[0])

        # print(img_id)
        # break
        # if img_id!=7:
        #     continue
        data={}
        img_filename = f'{ds_imgs_path}/{img_id}.jpg'
        

        for intrested_class,intrested_class_lbl in (zip(tqdm(intrested_class_ls, leave=False, desc='attributes : '),intrested_class_lss)):
            st_load = time.time()
            load_image_to_explain(intrested_class,img_id=img_id,bg_type=background_type)
            load_ground_truth(intrested_class_lss=[intrested_class_lbl],img_id=img_id)
            ####################################################
            # f_S_ones = f_masked(np.ones((1,224,224)).astype(bool))[0][0]
            # f_0_zeros = f_masked(np.zeros((1,224,224)).astype(bool))[0][0]
            ####################################################
            # load_ground_truth(intrested_class_lss,img_id=img_id)
            if first_exp_print:
                print(f'{img_id} | {classes_names[intrested_class]} | --> f_S: [{f_S}] f_0: [{f_0}]')
                # f_S_ones = f_masked(np.ones((1,224,224)).astype(bool))[0][0]
                # f_0_zeros = f_masked(np.zeros((1,224,224)).astype(bool))[0][0]
                # print('f_S_ones: ',f_S_ones)
                # print('f_0_zeros: ',f_0_zeros)
                first_exp_print = False
            # load_ground_truth(img_id)
            ###############################################################
            ground_truth_final = union_ground_truths[classes_names[intrested_class]]==255

            # plt.imshow(ground_truth_final)
            # plt.show()
            ###############################################################
            true_intrested_class = df_attr[df_attr.image_id==image_id][intrested_class_lbl].values[0]==1
            if intrested_class_lbl=='Brown_Hair':
                dominant_hair = get_real_class(image_id,df_attr,filter_hairs)
            else:
                dominant_hair = 'Eyeglasses?'
            ###############################################################
            invert_heatmap = (predicted_fS[intrested_class] < f_0)
            if verbose_print:
                print(f'{img_id} \t {classes_names[intrested_class]:<10}? == {not invert_heatmap}')
            
            # if abs(predicted_fS[intrested_class])<1: 
            #     if verbose_print:
            #         print(f'skipping - {img_id} - {classes_names[intrested_class]:<10}')
            #     continue
            
            if abs(f_S - f_0) < 0.1:
                print(f'Skipping Img: {image_id} as ', abs(f_S - f_0))
                continue

            ##################################################################################
            time_load = time.time()-st_load
            heatmaps, aucD, aucI,IoU = {}, {}, {},{}
            time_exp = {}
                        
            # for n,_,funct in tqdm(methods, leave=False, desc='XAI methods'):
            #     st = time.time(); 
            #     heatmaps[n] = funct()
            #     time_exp[n] = time.time()-st
            for n,_,funct in tqdm(methods, desc='Explanation',leave=False):
                heatmap_filename = f'{vector_path}//heatmaps_{image_id}_{n}.pkl'
                # st = time.time()
                # heatmaps[n] = funct()
                # time_exp[n] = time.time()-st
                
                if save_vectors_heatmaps:
                    if os.path.exists(heatmap_filename):
                        if verbose_print:
                            print(f'loaded : {image_id}_{n}.pkl')
                        st = time.time()
                        if n not in disallowed_vectors:
                            with open(heatmap_filename, 'rb') as rf:
                                heatmaps[n] = pickle.load(rf)
                        time_exp[n] = time.time()-st
                    else:
                        st = time.time()
                        heatmaps[n] = funct()
                        time_exp[n] = time.time()-st
                        # np.save(heatmap_filename,heatmaps[n], allow_pickle=True)
                        if n not in disallowed_vectors:
                            # print(n, 'is allowed')
                            with open(heatmap_filename, 'wb') as pf:
                                pickle.dump(heatmaps[n], pf)
                        else:
                            pass
                            # print(n, 'not allowed to save')
                else:
                    st = time.time()
                    heatmaps[n] = funct()
                    time_exp[n] = time.time()-st

            for n,_,funct in methods:

                # FOR NEGATIVE SHAPELY VALUES
                ######     INVERSION OF HEAT
                if invert_heatmap:
                    heatmaps[n]=-heatmaps[n]
                ###################################################################
                st = time.time(); 
                aucD[n] = saliency_to_auc(f_masked,heatmaps[n][0],f_S, f_0, predicted_cls=0, method='del', batch_size=batch_size)
                time_aucD = time.time()-st; 
        
                st = time.time(); 
                aucI[n] = saliency_to_auc(f_masked,heatmaps[n][0],f_S, f_0, predicted_cls=0, method='ins', batch_size=batch_size)
                time_aucI = time.time()-st; 
                
                st = time.time(); 
                IoU[n]  = calc_IoU_curve(ground_truth_final.flatten(), heatmaps[n][0].flatten())
                time_IoU = time.time()-st; 
            
            for n,_,funct in methods:
                st_plot = time.time();  
                #################################
                plt_ttl = f'img_id: {img_id}, explained_class: {intrested_class_lbl} '\
                            f'True Class: {dominant_hair} \n'\
                            f'f_S:\ {f_S:0.6}, f_0: {f_0:0.6}, delta: {abs(f_S-f_0):0.6}'

                if first_exp_plot:
                    plot_heatmaps(image_to_explain,ground_truth_final,heatmaps,save_dir=results_path,image_name=img_id,save_figs=True, destroy_fig=False, plot_title=True)
                    plot_IoU(save_dir=results_path,destroy_fig=False,image_name=img_id, save_figs=True)
                    fun_plot_performance(aucI,aucD,IoU,path=results_path, file_name=img_id,layout='2rows',
                                     set_title=True,ttl=plt_ttl)
                    #######
                    first_exp_plot = False
                    plot_heatmaps(image_to_explain,ground_truth_final,heatmaps,save_dir=results_path,image_name=img_id,save_figs=True, destroy_fig=True)
                    plot_IoU(save_dir=results_path,destroy_fig=True,image_name=img_id, save_figs=True)
                else:
                    plot_heatmaps(image_to_explain,ground_truth_final,heatmaps,save_dir=results_path,image_name=img_id,save_figs=True, destroy_fig=True)
                    plot_IoU(save_dir=results_path,destroy_fig=True,image_name=img_id, save_figs=True)
                time_plotting = time.time()-st_plot; 
                # fun_plot_performance(aucI,aucD,IoU,path=results_path, file_name=img_id, layout='2rows')
                # fun_plot_performance(aucI,aucD,IoU,path=results_path, file_name=img_id,layout='2rows',
                #                      set_title=True,ttl=plt_ttl)

                # return {'xs':xs, 'ms':ms, 'qs':qs, 
                #     'f_0':f_0, 'f_S':f_S, 'flipped':flipped, 
                #     'ys':ys, 'ysr':ys_rescaled,
                #     'y_clip':y_clipped, 'y_clipr':y_clipped_rescaled, 
                #     'y_adj':y_adjusted, 'y_adjr':y_adjusted_rescaled, 
                #     'method':method, 'predicted_cls':predicted_cls,
                #     'auc':auc, 'auc_r':auc_r,
                #     'auc_mae':auc_mae, 'auc_mse':auc_mse, 'auc_rmse':np.sqrt(auc_mse), 
                #     'auc_clip':auc_clip, 'auc_clipr':auc_clipr,
                #     'auc_adj':auc_adj, 'auc_adjr':auc_adjr}

                data = {'image'                : img_id,
                        'background_type'      : background_type,
                        'intrested_class'      : intrested_class,
                        'intrested_class_lbl'  : intrested_class_lbl,
                        'real_class'           : dominant_hair,
                        'true_intrested_class' : not invert_heatmap,
                        'f_S'                  : f_S,
                        'f_0'                  : f_0,
                        # 'f_G'                   : f_G,
                        # 'f_B'                   : f_B,
                        'delta_f'              : f_S-f_0,
                        'f_T'                  : np.sum(heatmaps[n][0]),
                        'f_N'                  : len(np.unique(heatmaps[n][0])),           ## UNIQUE PATCHES IN EXPLANATION
                        'method'               : n,

                        ##############################
                        'aucI_pred'     : aucI[n]['auc'],  # aucI[n][-3],
                        'aucD_pred'     : aucD[n]['auc'],  # aucD[n][-3],
                        ##############################
                        'aucI_r'     : aucI[n]['auc_r'],  # aucI[n][-3],
                        'aucD_r'     : aucD[n]['auc_r'],  # aucD[n][-3],
                        ###
                        'aucI_adj'     : aucI[n]['auc_adj'],  # aucI[n][-3],
                        'aucD_adj'     : aucD[n]['auc_adj'],  # aucD[n][-3],
                        ###
                        'aucI_adj_r'     : aucI[n]['auc_adjr'],  # aucI[n][-3],
                        'aucD_adj_r'     : aucD[n]['auc_adjr'],  # aucD[n][-3],
                        ###
                        'aucI_clip'       :aucI[n]['auc_clip'], 
                        'aucI_clipr'      :aucI[n]['auc_clipr'],
                        ###
                        'aucD_clip'       :aucD[n]['auc_clip'], 
                        'aucD_clipr'      :aucD[n]['auc_clipr'],
                        ###

                        ###
                        'threshold'            : IoU[n][2],
                        'best_point'           : IoU[n][3],
                        'max_IoU'              : np.max(IoU[n][1]),
                        'au_IoU'               : IoU[n][4],
                        #################################################
                        'time_exp'             : time_exp[n],
                        'time_aucI'            : time_aucI,
                        'time_aucD'            : time_aucD,
                        'time_IoU'             : time_IoU,
                        'time_plotting'        : time_plotting,    
                        'time_total'           : time_load+time_exp[n]+time_aucI+time_aucD+time_IoU+time_plotting,
                       }
                data_to_csv.append(data)
                df_IoU = pd.DataFrame(data_to_csv)
            df_IoU.to_csv(csv_filename)
            
        
        # break
    df_IoU.to_csv(csv_filename)
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

In [ ]:
print('Duration: {}'.format(end_time - start_time))
# print('csv file saved at ', csv_filename)

In [ ]:
df_IoU.head()

# END